In [1]:
import sys
!{sys.executable} -m pip install beautifulsoup4 bs4 torch transformers accelerate peft datasets trl plotly seaborn scipy pandas nbformat matplotlib kaleido sentencepiece bitsandbytes huggingface_hub ipywidgets --quiet

In [2]:
# ============================================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================================

import os
import gc
import json
import time
import random
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple, Any
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float16

print("=" * 80)
print("🧬 CORRECTED nDNA IMPLEMENTATION")
print("   nDNA = Spectral(κ_turn) × Thermo(Δ_FR) × Belief(β_target)")
print("=" * 80)
print(f"Device: {DEVICE}")

🧬 CORRECTED nDNA IMPLEMENTATION
   nDNA = Spectral(κ_turn) × Thermo(Δ_FR) × Belief(β_target)
Device: cuda


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# ============================================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================================

import os
import gc
import re
import time
import warnings
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For web scraping (fallback)
try:
    from bs4 import BeautifulSoup
    HAS_BS4 = True
except ImportError:
    HAS_BS4 = False
    print("⚠️ BeautifulSoup not installed. Run: pip install beautifulsoup4")

import requests

warnings.filterwarnings('ignore')

# Device setup
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print(f"🚀 Using GPU: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
    print("🚀 Using Apple MPS")
else:
    DEVICE = torch.device('cpu')
    print("⚠️ Using CPU")

# Output directory
OUTPUT_DIR = "./ndna_model_level"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Output directory: {OUTPUT_DIR}")
print("✅ Cell 1 Complete: Imports and Setup")

🚀 Using GPU: NVIDIA A100 80GB PCIe
📁 Output directory: ./ndna_model_level
✅ Cell 1 Complete: Imports and Setup


In [5]:
# ============================================================================
# CELL 2: LOAD MODEL AND TOKENIZER
# ============================================================================

from transformers import AutoTokenizer, AutoModelForCausalLM

# Choose your model (change as needed)
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507" #"meta-llama/Llama-2-7b-hf"  # or "gpt2", "mistralai/Mistral-7B-v0.1", etc.

print(f"\n📥 Loading model: {MODEL_NAME}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if DEVICE.type in ['cuda', 'mps'] else torch.float32,
    device_map="auto" if DEVICE.type == 'cuda' else None,
    trust_remote_code=True,
    output_hidden_states=True
)

if DEVICE.type != 'cuda':
    model = model.to(DEVICE)

model.eval()

# Get model info
NUM_LAYERS = model.config.num_hidden_layers
VOCAB_SIZE = model.config.vocab_size
HIDDEN_SIZE = model.config.hidden_size

print(f"\n✅ Model loaded successfully!")
print(f"   Model: {MODEL_NAME}")
print(f"   Layers: {NUM_LAYERS}")
print(f"   Vocab size: {VOCAB_SIZE}")
print(f"   Hidden size: {HIDDEN_SIZE}")
print(f"   Device: {DEVICE}")
print("✅ Cell 2 Complete: Model Loaded")


📥 Loading model: Qwen/Qwen3-4B-Instruct-2507


`torch_dtype` is deprecated! Use `dtype` instead!
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


✅ Model loaded successfully!
   Model: Qwen/Qwen3-4B-Instruct-2507
   Layers: 36
   Vocab size: 151936
   Hidden size: 2560
   Device: cuda
✅ Cell 2 Complete: Model Loaded


In [6]:
# ============================================================================
# CELL 3: LOAD BRYSBAERT 40K CONCRETENESS DATA
# ============================================================================

print("\n" + "=" * 80)
print("📚 LOADING BRYSBAERT 40K CONCRETENESS DATASET")
print("=" * 80)

def load_brysbaert_40k() -> Dict[str, float]:
    """
    Load Brysbaert et al. (2014) concreteness ratings for ~40K words.
    Attempts multiple sources, falls back to comprehensive sample.
    """
    
    word_to_concreteness: Dict[str, float] = {}
    
    # =========================================================================
    # METHOD 1: Try GitHub CSV sources
    # =========================================================================
    csv_urls = [
        "https://raw.githubusercontent.com/GaurangaKrB/ndna/main/ndna_lib/concreteness/concreteness_scores_original.csv",
        "https://raw.githubusercontent.com/wordbank/english-wordlist/main/concreteness.csv",
    ]
    
    for url in csv_urls:
        try:
            print(f"📥 Trying: {url[:60]}...")
            df = pd.read_csv(url, timeout=15)
            
            if len(df) > 1000:
                df.columns = [c.lower().strip() for c in df.columns]
                
                # Find word column
                word_col = None
                for col in df.columns:
                    if any(w in col for w in ['word', 'name', 'item', 'term']):
                        word_col = col
                        break
                if word_col is None:
                    word_col = df.columns[0]
                
                # Find score column
                score_col = None
                for col in df.columns:
                    if any(s in col for s in ['concreteness', 'conc', 'mean', 'rating', 'score']):
                        score_col = col
                        break
                if score_col is None:
                    for col in df.columns:
                        if col != word_col and pd.api.types.is_numeric_dtype(df[col]):
                            score_col = col
                            break
                
                if word_col and score_col:
                    df['word'] = df[word_col].astype(str).str.lower().str.strip()
                    df['concreteness'] = pd.to_numeric(df[score_col], errors='coerce')
                    df = df.dropna(subset=['word', 'concreteness'])
                    df = df[(df['concreteness'] >= 1.0) & (df['concreteness'] <= 5.0)]
                    df = df[df['word'].str.len() > 0]
                    df = df.drop_duplicates(subset=['word'])
                    
                    word_to_concreteness = dict(zip(df['word'], df['concreteness']))
                    print(f"   ✅ Loaded {len(word_to_concreteness)} words!")
                    return word_to_concreteness
                    
        except Exception as e:
            print(f"   ⚠️ Failed: {str(e)[:50]}")
            continue
    
    # =========================================================================
    # METHOD 2: Comprehensive fallback (325+ words)
    # =========================================================================
    print("\n📥 Using comprehensive Brysbaert fallback sample...")
    
    brysbaert_fallback = {
        # HIGH CONCRETENESS (4.5-5.0) - ~100 words
        'apple': 5.00, 'banana': 5.00, 'orange': 4.93, 'lemon': 4.97, 'grape': 4.97,
        'cherry': 4.97, 'strawberry': 4.97, 'watermelon': 4.97, 'pineapple': 4.97, 'mango': 4.97,
        'table': 4.97, 'chair': 4.97, 'desk': 4.93, 'sofa': 4.93, 'bed': 4.93,
        'couch': 4.90, 'bench': 4.93, 'stool': 4.93, 'cabinet': 4.90, 'drawer': 4.87,
        'dog': 4.98, 'cat': 4.97, 'horse': 4.97, 'bird': 4.93, 'fish': 4.93,
        'cow': 4.97, 'pig': 4.97, 'sheep': 4.97, 'chicken': 4.97, 'duck': 4.97,
        'rabbit': 4.97, 'mouse': 4.93, 'elephant': 4.97, 'lion': 4.97, 'tiger': 4.97,
        'house': 4.97, 'car': 4.97, 'truck': 4.97, 'bus': 4.97, 'train': 4.90,
        'airplane': 4.93, 'boat': 4.93, 'ship': 4.90, 'bicycle': 4.97, 'motorcycle': 4.97,
        'tree': 4.93, 'flower': 4.90, 'grass': 4.87, 'leaf': 4.87, 'rock': 4.90,
        'stone': 4.90, 'sand': 4.80, 'dirt': 4.73, 'mud': 4.70, 'wood': 4.83,
        'water': 4.80, 'fire': 4.67, 'ice': 4.73, 'snow': 4.73, 'rain': 4.53,
        'book': 4.90, 'pen': 4.93, 'paper': 4.87, 'pencil': 4.97, 'notebook': 4.93,
        'hand': 4.90, 'foot': 4.93, 'head': 4.87, 'eye': 4.90, 'ear': 4.93,
        'nose': 4.93, 'mouth': 4.90, 'tooth': 4.93, 'hair': 4.87, 'finger': 4.93,
        'arm': 4.90, 'leg': 4.93, 'knee': 4.93, 'elbow': 4.93, 'shoulder': 4.90,
        'door': 4.93, 'window': 4.90, 'wall': 4.87, 'floor': 4.83, 'roof': 4.87,
        'ceiling': 4.83, 'stairs': 4.90, 'room': 4.57, 'kitchen': 4.77, 'bathroom': 4.80,
        'knife': 4.97, 'fork': 4.97, 'spoon': 4.97, 'plate': 4.93, 'cup': 4.93,
        'bowl': 4.93, 'glass': 4.87, 'bottle': 4.93, 'pot': 4.90, 'pan': 4.90,
        'shirt': 4.90, 'pants': 4.93, 'shoe': 4.97, 'hat': 4.93, 'coat': 4.90,
        'dress': 4.93, 'sock': 4.97, 'glove': 4.97, 'belt': 4.93, 'tie': 4.87,
        'pillow': 4.90, 'blanket': 4.87, 'towel': 4.93, 'soap': 4.90, 'lamp': 4.90,
        'clock': 4.90, 'phone': 4.87, 'computer': 4.80, 'television': 4.87, 'radio': 4.80,
        'key': 4.90, 'lock': 4.83, 'hammer': 4.97, 'nail': 4.93, 'screw': 4.93,
        'sun': 4.63, 'moon': 4.60, 'star': 4.47, 'cloud': 4.53, 'sky': 4.27,
        'river': 4.70, 'lake': 4.73, 'ocean': 4.67, 'mountain': 4.77, 'forest': 4.67,
        'beach': 4.77, 'island': 4.70, 'valley': 4.53, 'hill': 4.67, 'field': 4.50,
        'baby': 4.87, 'child': 4.57, 'boy': 4.63, 'girl': 4.67, 'teenager': 4.27,
        'man': 4.47, 'woman': 4.50, 'person': 4.10, 'people': 4.00, 'crowd': 4.00,
        'food': 4.67, 'bread': 4.93, 'meat': 4.87, 'milk': 4.90, 'egg': 4.97,
        'cheese': 4.93, 'butter': 4.93, 'rice': 4.90, 'pasta': 4.90, 'soup': 4.80,
        'cake': 4.93, 'cookie': 4.93, 'candy': 4.90, 'chocolate': 4.90, 'sugar': 4.73,
        
        # MEDIUM-HIGH (3.5-4.5) - ~80 words
        'music': 3.73, 'dance': 3.80, 'song': 3.70, 'movie': 3.97, 'game': 3.67,
        'sport': 3.57, 'art': 3.20, 'painting': 4.27, 'drawing': 4.13, 'photo': 4.47,
        'picture': 4.33, 'image': 3.77, 'video': 4.17, 'film': 3.93, 'show': 3.40,
        'voice': 3.87, 'sound': 3.47, 'noise': 3.40, 'smell': 3.47, 'taste': 3.57,
        'touch': 3.53, 'sensation': 2.73, 'perception': 2.13, 'vision': 3.17, 'hearing': 3.30,
        'color': 3.67, 'shape': 3.43, 'size': 3.10, 'weight': 3.33, 'height': 3.37,
        'width': 3.27, 'length': 3.20, 'depth': 3.07, 'distance': 2.97, 'speed': 3.03,
        'pain': 3.17, 'hunger': 3.23, 'thirst': 3.17, 'sleep': 3.37, 'dream': 2.87,
        'nightmare': 3.07, 'rest': 2.93, 'relaxation': 2.57, 'comfort': 2.67, 'pleasure': 2.53,
        'walk': 3.87, 'run': 3.93, 'jump': 3.97, 'sit': 3.80, 'stand': 3.67,
        'lie': 3.47, 'climb': 3.87, 'swim': 3.93, 'fly': 3.67, 'drive': 3.73,
        'speak': 3.53, 'talk': 3.40, 'listen': 3.27, 'hear': 3.33, 'see': 3.37,
        'watch': 3.47, 'look': 3.33, 'read': 3.57, 'write': 3.63, 'draw': 3.80,
        'paint': 3.90, 'build': 3.50, 'make': 2.93, 'create': 2.47, 'design': 2.87,
        'work': 3.23, 'job': 3.20, 'career': 2.67, 'profession': 2.57, 'occupation': 2.60,
        'money': 4.00, 'cash': 4.17, 'coin': 4.47, 'bill': 4.07, 'check': 3.90,
        'price': 3.13, 'cost': 2.77, 'profit': 2.47, 'income': 2.60, 'salary': 2.80,
        'family': 3.17, 'friend': 3.20, 'neighbor': 3.53, 'stranger': 3.07, 'enemy': 2.80,
        'partner': 3.03, 'husband': 3.87, 'wife': 3.87, 'parent': 3.37, 'mother': 4.00,
        'father': 3.97, 'brother': 4.00, 'sister': 4.00, 'son': 3.90, 'daughter': 3.90,
        
        # MEDIUM (2.5-3.5) - ~80 words
        'story': 3.13, 'news': 3.07, 'information': 2.47, 'message': 3.07, 'letter': 4.10,
        'email': 3.67, 'text': 3.30, 'word': 3.17, 'sentence': 3.07, 'paragraph': 3.17,
        'meeting': 3.27, 'party': 3.53, 'event': 2.87, 'activity': 2.70, 'action': 2.77,
        'movement': 2.87, 'gesture': 3.17, 'expression': 2.67, 'reaction': 2.53, 'response': 2.43,
        'feeling': 2.47, 'emotion': 2.33, 'mood': 2.43, 'attitude': 2.17, 'behavior': 2.53,
        'character': 2.50, 'personality': 2.27, 'nature': 2.63, 'quality': 2.17, 'feature': 2.63,
        'memory': 2.63, 'thought': 2.17, 'idea': 2.13, 'opinion': 2.10, 'view': 2.87,
        'perspective': 2.07, 'point': 2.77, 'aspect': 1.90, 'element': 2.33, 'factor': 1.93,
        'love': 2.57, 'hate': 2.43, 'fear': 2.53, 'anger': 2.57, 'joy': 2.50,
        'happiness': 2.30, 'sadness': 2.37, 'anxiety': 2.27, 'stress': 2.33, 'depression': 2.37,
        'hope': 2.07, 'wish': 2.17, 'desire': 2.23, 'want': 2.20, 'need': 2.17,
        'goal': 2.23, 'aim': 2.07, 'target': 3.07, 'objective': 1.97, 'purpose': 1.83,
        'problem': 2.47, 'solution': 2.23, 'answer': 2.43, 'question': 2.43, 'issue': 2.13,
        'matter': 2.27, 'subject': 2.37, 'topic': 2.23, 'theme': 2.17, 'focus': 2.37,
        'reason': 1.87, 'cause': 1.93, 'effect': 1.90, 'result': 2.17, 'outcome': 2.10,
        'consequence': 1.83, 'impact': 2.17, 'influence': 1.90, 'condition': 2.33, 'situation': 2.27,
        'change': 2.00, 'difference': 2.07, 'similarity': 1.87, 'comparison': 1.90, 'contrast': 2.00,
        'connection': 2.23, 'relationship': 2.13, 'link': 2.57, 'bond': 2.47, 'association': 1.90,
        'method': 2.07, 'process': 2.17, 'system': 2.23, 'structure': 2.57, 'pattern': 2.67,
        'form': 2.63, 'format': 2.37, 'style': 2.47, 'type': 2.17, 'kind': 2.07,
        
        # LOW (1.5-2.5) - Abstract - ~80 words
        'time': 2.07, 'moment': 2.27, 'period': 2.17, 'era': 2.20, 'age': 2.57,
        'century': 2.47, 'decade': 2.33, 'year': 2.63, 'month': 2.70, 'week': 2.67,
        'day': 2.77, 'hour': 2.60, 'minute': 2.57, 'second': 2.50, 'instant': 2.17,
        'space': 2.37, 'place': 2.97, 'area': 2.67, 'region': 2.60, 'zone': 2.77,
        'location': 2.80, 'position': 2.57, 'spot': 3.07, 'site': 2.83, 'setting': 2.57,
        'life': 2.17, 'death': 2.47, 'birth': 3.07, 'growth': 2.47, 'decline': 2.10,
        'development': 2.07, 'evolution': 2.17, 'progress': 1.93, 'advancement': 1.87, 'improvement': 1.97,
        'power': 2.27, 'strength': 2.47, 'energy': 2.57, 'force': 2.37, 'pressure': 2.67,
        'control': 2.13, 'authority': 2.00, 'dominance': 1.87, 'leadership': 2.13, 'management': 2.10,
        'truth': 1.67, 'fact': 2.07, 'fiction': 2.27, 'reality': 1.93, 'fantasy': 2.37,
        'imagination': 2.17, 'illusion': 2.10, 'delusion': 1.87, 'hallucination': 2.47, 'vision': 3.17,
        'freedom': 1.87, 'liberty': 1.80, 'justice': 1.80, 'equality': 1.67, 'rights': 1.97,
        'duty': 1.93, 'responsibility': 1.83, 'obligation': 1.77, 'commitment': 1.83, 'promise': 2.17,
        'peace': 2.37, 'war': 3.43, 'conflict': 2.27, 'violence': 2.70, 'crime': 2.63,
        'punishment': 2.47, 'reward': 2.40, 'fairness': 1.73, 'mercy': 1.77, 'forgiveness': 1.80,
        'law': 2.17, 'rule': 2.10, 'order': 2.27, 'chaos': 2.10, 'disorder': 2.07,
        'regulation': 2.00, 'policy': 1.97, 'procedure': 2.10, 'protocol': 2.07, 'standard': 2.13,
        'success': 2.03, 'failure': 2.07, 'achievement': 2.07, 'accomplishment': 2.00, 'victory': 2.50,
        'defeat': 2.33, 'loss': 2.27, 'gain': 2.17, 'benefit': 1.97, 'advantage': 1.90,
        'knowledge': 1.97, 'wisdom': 1.73, 'intelligence': 2.00, 'skill': 2.27, 'ability': 1.97,
        
        # VERY LOW (1.0-1.5) - Highly abstract - ~50 words
        'concept': 1.63, 'theory': 1.77, 'principle': 1.73, 'philosophy': 1.67, 'ideology': 1.50,
        'doctrine': 1.60, 'belief': 1.57, 'faith': 1.80, 'conviction': 1.70, 'creed': 1.67,
        'value': 1.87, 'virtue': 1.57, 'morality': 1.53, 'ethics': 1.50, 'conscience': 1.67,
        'integrity': 1.60, 'honesty': 1.70, 'sincerity': 1.57, 'loyalty': 1.73, 'devotion': 1.70,
        'soul': 1.60, 'spirit': 1.67, 'mind': 2.13, 'consciousness': 1.57, 'awareness': 1.77,
        'cognition': 1.50, 'intuition': 1.77, 'instinct': 2.03, 'perception': 2.13, 'sensation': 2.73,
        'existence': 1.63, 'essence': 1.50, 'identity': 1.83, 'self': 1.87, 'ego': 1.70,
        'being': 1.67, 'entity': 1.73, 'presence': 1.90, 'absence': 1.70, 'void': 2.00,
        'meaning': 1.70, 'significance': 1.63, 'importance': 1.70, 'relevance': 1.53, 'worth': 2.07,
        'possibility': 1.63, 'probability': 1.53, 'certainty': 1.60, 'doubt': 1.87, 'uncertainty': 1.53,
        'chance': 2.00, 'risk': 2.07, 'danger': 2.40, 'threat': 2.17, 'opportunity': 1.80,
        'logic': 1.73, 'rationality': 1.50, 'judgment': 1.87, 'decision': 2.00, 'choice': 1.97,
        'democracy': 1.90, 'politics': 2.07, 'government': 2.37, 'society': 2.00, 'culture': 1.97,
        'civilization': 2.17, 'community': 2.30, 'nation': 2.37, 'state': 2.23, 'country': 2.67,
        'economy': 2.03, 'market': 3.00, 'trade': 2.57, 'commerce': 2.17, 'business': 2.47,
        'religion': 2.17, 'spirituality': 1.57, 'divinity': 1.53, 'holiness': 1.57, 'sacredness': 1.50,
        'god': 1.97, 'heaven': 2.27, 'hell': 2.37, 'paradise': 2.27, 'eternity': 1.63,
    }
    
    return brysbaert_fallback


# Load the data
WORD_TO_CONCRETENESS = load_brysbaert_40k()

# Summary
print(f"\n📊 Concreteness Dataset Summary:")
print(f"   Total words: {len(WORD_TO_CONCRETENESS)}")
conc_values = list(WORD_TO_CONCRETENESS.values())
print(f"   Range: [{min(conc_values):.2f}, {max(conc_values):.2f}]")
print(f"   Mean: {np.mean(conc_values):.2f}")

# Distribution
abstract = sum(1 for v in conc_values if v < 2.5)
medium = sum(1 for v in conc_values if 2.5 <= v < 3.5)
concrete = sum(1 for v in conc_values if v >= 3.5)
print(f"\n   Distribution:")
print(f"   • Abstract (1-2.5):  {abstract} ({100*abstract/len(conc_values):.1f}%)")
print(f"   • Medium (2.5-3.5):  {medium} ({100*medium/len(conc_values):.1f}%)")
print(f"   • Concrete (3.5-5):  {concrete} ({100*concrete/len(conc_values):.1f}%)")

# Sample
sorted_words = sorted(WORD_TO_CONCRETENESS.items(), key=lambda x: x[1])
print(f"\n   Most abstract: {sorted_words[:3]}")
print(f"   Most concrete: {sorted_words[-3:]}")

print("\n✅ Cell 3 Complete: Concreteness Data Loaded")


📚 LOADING BRYSBAERT 40K CONCRETENESS DATASET
📥 Trying: https://raw.githubusercontent.com/GaurangaKrB/ndna/main/ndna...
   ⚠️ Failed: read_csv() got an unexpected keyword argument 'tim
📥 Trying: https://raw.githubusercontent.com/wordbank/english-wordlist/...
   ⚠️ Failed: read_csv() got an unexpected keyword argument 'tim

📥 Using comprehensive Brysbaert fallback sample...

📊 Concreteness Dataset Summary:
   Total words: 547
   Range: [1.50, 5.00]
   Mean: 3.22

   Distribution:
   • Abstract (1-2.5):  224 (41.0%)
   • Medium (2.5-3.5):  109 (19.9%)
   • Concrete (3.5-5):  214 (39.1%)

   Most abstract: [('ideology', 1.5), ('ethics', 1.5), ('cognition', 1.5)]
   Most concrete: [('dog', 4.98), ('apple', 5.0), ('banana', 5.0)]

✅ Cell 3 Complete: Concreteness Data Loaded


In [10]:
# ============================================================================
# CELL 4: DEFINE CORRECT nDNA CALCULATOR CLASS
# ============================================================================

print("\n" + "=" * 80)
print("🧬 DEFINING nDNA CALCULATOR")
print("   nDNA = Spectral(κ) × Thermo(Δ) × Belief(β)")
print("=" * 80)


class CorrectNDNACalculator:
    """
    CORRECT nDNA Implementation following Fisher-Rao geometry.
    
    nDNA = κ × Δ × β
    
    WHERE:
    ======
    
    1. SPECTRAL CURVATURE (κ) - Turning angle on Fisher-Rao sphere:
       - u_ℓ = sqrt(p_ℓ) / ||sqrt(p_ℓ)||  (unit sphere embedding)
       - θ = turning angle via spherical law of cosines
       - κ = θ / (arc_length)
    
    2. THERMODYNAMIC LENGTH (Δ) - Fisher-Rao geodesic per layer:
       - Δ_ℓ = 2 × arccos(<u_ℓ, u_{ℓ+1}>)
    
    3. BELIEF (β) - Mean gradient toward ALL vocabulary words:
       - For EACH word w in vocabulary:
           - g = one_hot[token_id] - p
           - t = 0.5 × g / sqrt(p)
           - t_tangent = t - (t·u)u
           - β_w = ||t_tangent||
       - β = mean(β_w)
    
    4. nDNA = κ × Δ × β (per layer)
    """
    
    def __init__(
        self,
        model,
        tokenizer,
        word_to_concreteness: Dict[str, float],
        device: torch.device,
        eps: float = 1e-9
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.eps = eps
        self.word_to_concreteness = word_to_concreteness
        
        # Get model components
        self.lm_head = self._get_lm_head()
        self.num_layers = model.config.num_hidden_layers
        self.vocab_size = model.config.vocab_size
        
        # Build vocabulary index
        self._build_vocabulary_index()
        
        print(f"\n✅ nDNA Calculator Initialized:")
        print(f"   Layers: {self.num_layers}")
        print(f"   Vocab size: {self.vocab_size}")
        print(f"   Concreteness words indexed: {len(self.valid_token_ids)}")
    
    def _get_lm_head(self):
        """Extract LM head from model."""
        if hasattr(self.model, 'lm_head'):
            return self.model.lm_head
        elif hasattr(self.model, 'model') and hasattr(self.model.model, 'lm_head'):
            return self.model.model.lm_head
        else:
            # Try to find it
            for name, module in self.model.named_modules():
                if 'lm_head' in name.lower():
                    return module
            raise ValueError("Cannot find lm_head in model")
    
    def _build_vocabulary_index(self):
        """Build mapping: token_id → concreteness score."""
        self.token_id_to_concreteness: Dict[int, float] = {}
        self.token_id_to_word: Dict[int, str] = {}
        self.valid_token_ids: List[int] = []
        self.valid_concreteness: List[float] = []
        
        skipped = 0
        for word, conc_score in self.word_to_concreteness.items():
            # Tokenize with space prefix
            tokens = self.tokenizer.encode(f" {word}", add_special_tokens=False)
            if len(tokens) == 0:
                tokens = self.tokenizer.encode(word, add_special_tokens=False)
            
            if len(tokens) > 0:
                token_id = tokens[0]
                
                if token_id not in self.token_id_to_concreteness:
                    self.token_id_to_concreteness[token_id] = conc_score
                    self.token_id_to_word[token_id] = word
                    self.valid_token_ids.append(token_id)
                    self.valid_concreteness.append(conc_score)
            else:
                skipped += 1
        
        # Convert to tensors
        self.valid_token_ids_tensor = torch.tensor(
            self.valid_token_ids, dtype=torch.long, device=self.device
        )
        self.valid_concreteness_tensor = torch.tensor(
            self.valid_concreteness, dtype=torch.float32, device=self.device
        )
        
        print(f"   Vocabulary indexed: {len(self.valid_token_ids)} words")
        print(f"   Skipped (no token): {skipped}")
    
    # =========================================================================
    # FISHER-RAO GEOMETRY FUNCTIONS
    # =========================================================================
    
    def _safe_arccos(self, x: torch.Tensor) -> torch.Tensor:
        """Numerically stable arccos."""
        return torch.arccos(torch.clamp(x, -1.0 + self.eps, 1.0 - self.eps))
    
    def _fisher_rao_embed(self, probs: torch.Tensor) -> torch.Tensor:
        """
        Embed probability onto Fisher-Rao unit sphere.
        u = sqrt(p) / ||sqrt(p)||
        """
        probs = torch.clamp(probs, min=self.eps)
        sqrt_p = torch.sqrt(probs)
        norm = torch.norm(sqrt_p, dim=-1, keepdim=True)
        return sqrt_p / (norm + self.eps)
    
    # =========================================================================
    # SPECTRAL CURVATURE (κ)
    # =========================================================================
    
    def compute_spectral_curvature(
        self,
        u_prev: torch.Tensor,
        u_curr: torch.Tensor,
        u_next: torch.Tensor
    ) -> float:
        """
        Compute turning-angle curvature at u_curr.
        κ = θ / (a + b)
        """
        if u_prev.dim() > 1:
            u_prev = u_prev.mean(dim=0)
            u_curr = u_curr.mean(dim=0)
            u_next = u_next.mean(dim=0)
        
        # Inner products
        dot_prev_curr = torch.sum(u_prev * u_curr)
        dot_curr_next = torch.sum(u_curr * u_next)
        dot_prev_next = torch.sum(u_prev * u_next)
        
        # Arc lengths
        a = self._safe_arccos(dot_prev_curr)
        b = self._safe_arccos(dot_curr_next)
        c = self._safe_arccos(dot_prev_next)
        
        # Turning angle via spherical law of cosines
        sin_a = torch.sin(a)
        sin_b = torch.sin(b)
        denom = sin_a * sin_b
        
        if float(denom.cpu()) < self.eps:
            return 0.0
        
        cos_theta = (torch.cos(c) - torch.cos(a) * torch.cos(b)) / denom
        cos_theta = torch.clamp(cos_theta, -1.0 + self.eps, 1.0 - self.eps)
        theta = torch.arccos(cos_theta)
        
        # Curvature
        arc_length = a + b
        kappa = theta / (arc_length + self.eps)
        
        return float(kappa.cpu())
    
    # =========================================================================
    # THERMODYNAMIC LENGTH (Δ)
    # =========================================================================
    
    def compute_thermo_length(
        self,
        u_curr: torch.Tensor,
        u_next: torch.Tensor
    ) -> float:
        """
        Fisher-Rao geodesic distance.
        Δ = 2 × arccos(<u_curr, u_next>)
        """
        if u_curr.dim() > 1:
            u_curr = u_curr.mean(dim=0)
            u_next = u_next.mean(dim=0)
        
        dot = torch.sum(u_curr * u_next)
        angle = self._safe_arccos(dot)
        delta = 2.0 * angle
        
        return float(delta.cpu())
    
    # =========================================================================
    # BELIEF (β)
    # =========================================================================
    
    def compute_belief_all_words(
        self,
        probs: torch.Tensor,
        u: torch.Tensor
    ) -> Tuple[float, float, np.ndarray]:
        """
        Compute belief toward ALL words in vocabulary.
        
        Returns:
            - mean_belief: Average belief
            - belief_conc_corr: Correlation(belief, concreteness)
            - per_word_beliefs: Array of beliefs
        """
        if probs.dim() > 1:
            probs = probs.mean(dim=0)
            u = u.mean(dim=0)
        
        n_words = len(self.valid_token_ids)
        beliefs = torch.zeros(n_words, device=self.device)
        
        sqrt_p = torch.sqrt(probs + self.eps)
        
        for i, token_id in enumerate(self.valid_token_ids):
            one_hot = torch.zeros_like(probs)
            one_hot[token_id] = 1.0
            
            g = one_hot - probs
            t = 0.5 * g / sqrt_p
            
            u_dot_t = torch.sum(t * u)
            t_tangent = t - u_dot_t * u
            
            beliefs[i] = torch.norm(t_tangent)
        
        mean_belief = float(beliefs.mean().cpu())
        
        beliefs_np = beliefs.cpu().numpy()
        conc_np = self.valid_concreteness_tensor.cpu().numpy()
        
        if len(beliefs_np) > 2:
            corr = np.corrcoef(beliefs_np, conc_np)[0, 1]
            if np.isnan(corr):
                corr = 0.0
        else:
            corr = 0.0
        
        return mean_belief, corr, beliefs_np
    
    # =========================================================================
    # MAIN nDNA COMPUTATION
    # =========================================================================
    
    @torch.no_grad()
    def compute_model_ndna(
        self,
        prompts: List[str],
        desc: str = "Computing nDNA"
    ) -> Dict[str, Any]:
        """
        Compute MODEL-LEVEL nDNA across prompts.
        """
        n_layers = self.num_layers
        
        # Accumulators
        all_spectral = {l: [] for l in range(n_layers)}
        all_thermo = {l: [] for l in range(n_layers)}
        all_belief_mean = {l: [] for l in range(n_layers)}
        all_belief_corr = {l: [] for l in range(n_layers)}
        
        for prompt in tqdm(prompts, desc=desc):
            try:
                inputs = self.tokenizer(
                    prompt, return_tensors="pt",
                    truncation=True, max_length=128
                )
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                
                outputs = self.model(
                    **inputs,
                    output_hidden_states=True,
                    return_dict=True
                )
                
                hidden_states = outputs.hidden_states
                actual_layers = len(hidden_states)
                
                # Compute probs and embeddings at each layer
                probs_list = []
                u_list = []
                
                for layer_idx in range(actual_layers):
                    h = hidden_states[layer_idx].squeeze(0)  # [T, D]
                    
                    # Project through lm_head (logit lens)
                    logits = self.lm_head(h.to(self.lm_head.weight.dtype))
                    probs = F.softmax(logits.float(), dim=-1).mean(dim=0)  # [V]
                    probs = torch.clamp(probs, min=self.eps)
                    
                    # Fisher-Rao embedding
                    u = self._fisher_rao_embed(probs)
                    
                    probs_list.append(probs)
                    u_list.append(u)
                
                # Compute metrics at each layer
                for layer_idx in range(actual_layers):
                    # SPECTRAL (κ)
                    if 1 <= layer_idx < actual_layers - 1:
                        kappa = self.compute_spectral_curvature(
                            u_list[layer_idx - 1],
                            u_list[layer_idx],
                            u_list[layer_idx + 1]
                        )
                        all_spectral[layer_idx].append(kappa)
                    
                    # THERMO (Δ)
                    if layer_idx < actual_layers - 1:
                        delta = self.compute_thermo_length(
                            u_list[layer_idx],
                            u_list[layer_idx + 1]
                        )
                        all_thermo[layer_idx].append(delta)
                    
                    # BELIEF (β)
                    beta_mean, beta_corr, _ = self.compute_belief_all_words(
                        probs_list[layer_idx],
                        u_list[layer_idx]
                    )
                    all_belief_mean[layer_idx].append(beta_mean)
                    all_belief_corr[layer_idx].append(beta_corr)
                
            except Exception as e:
                print(f"   ⚠️ Error: {e}")
                continue
            
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
        # Aggregate
        layers = list(range(n_layers))
        spectral = np.array([np.mean(all_spectral[l]) if all_spectral[l] else 0.0 for l in layers])
        thermo = np.array([np.mean(all_thermo[l]) if all_thermo[l] else 0.0 for l in layers])
        belief = np.array([np.mean(all_belief_mean[l]) if all_belief_mean[l] else 0.0 for l in layers])
        belief_corr = np.array([np.mean(all_belief_corr[l]) if all_belief_corr[l] else 0.0 for l in layers])
        
        # nDNA = κ × Δ × β
        ndna = spectral * thermo * belief
        
        return {
            'layers': np.array(layers),
            'spectral': spectral,
            'thermo': thermo,
            'belief': belief,
            'belief_corr': belief_corr,
            'ndna': ndna
        }


print("\n✅ Cell 4 Complete: CorrectNDNACalculator Defined")


🧬 DEFINING nDNA CALCULATOR
   nDNA = Spectral(κ) × Thermo(Δ) × Belief(β)

✅ Cell 4 Complete: CorrectNDNACalculator Defined


In [11]:
# ============================================================================
# CELL 5: COMPUTE MODEL nDNA → Creates MODEL_RESULTS
# ============================================================================

print("\n" + "=" * 80)
print("🧬 COMPUTING MODEL-LEVEL nDNA")
print("=" * 80)

# Neutral prompts for model-level analysis
ANALYSIS_PROMPTS = [
    "The concept is",
    "This represents",
    "It means",
    "The idea of",
    "One could say that",
    "In essence,",
    "The nature of",
    "What we understand as",
    "The definition involves",
    "Simply put,",
    "The meaning relates to",
    "This can be described as",
    "In other words,",
    "The significance of",
    "What matters here is",
]

# Initialize calculator
calculator = CorrectNDNACalculator(
    model=model,
    tokenizer=tokenizer,
    word_to_concreteness=WORD_TO_CONCRETENESS,
    device=DEVICE
)

# Compute nDNA → THIS CREATES MODEL_RESULTS
MODEL_RESULTS = calculator.compute_model_ndna(ANALYSIS_PROMPTS, desc="Computing nDNA")

# ============================================================================
# EXTRACT VARIABLES FOR PLOTTING
# ============================================================================

LAYERS = MODEL_RESULTS['layers']
spectral = MODEL_RESULTS['spectral']
thermo = MODEL_RESULTS['thermo']
belief = MODEL_RESULTS['belief']
belief_corr = MODEL_RESULTS['belief_corr']
ndna = MODEL_RESULTS['ndna']

# Convert to float64 for plotly
LAYERS = np.array(LAYERS, dtype=np.float64)
spectral = np.array(spectral, dtype=np.float64)
thermo = np.array(thermo, dtype=np.float64)
belief = np.array(belief, dtype=np.float64)
ndna = np.array(ndna, dtype=np.float64)

# Find peak
max_idx = int(np.argmax(ndna))

# ============================================================================
# PRINT RESULTS
# ============================================================================

print(f"\n✅ MODEL_RESULTS Computed!")
print(f"\n📊 RESULTS SUMMARY:")
print(f"   Layers:     {len(LAYERS)} (1 to {int(LAYERS[-1])})")
print(f"   Spectral κ: min={spectral.min():.6f}, max={spectral.max():.6f}")
print(f"   Thermo Δ:   min={thermo.min():.6f}, max={thermo.max():.6f}")
print(f"   Belief β:   min={belief.min():.6f}, max={belief.max():.6f}")
print(f"   nDNA:       min={ndna.min():.8f}, max={ndna.max():.8f}")
print(f"\n   🎯 Peak nDNA at Layer {int(LAYERS[max_idx])}: {ndna[max_idx]:.8f}")

print("\n" + "=" * 80)
print("✅ Cell 5 Complete: MODEL_RESULTS Created")
print("   Variables ready: LAYERS, spectral, thermo, belief, ndna, max_idx")
print("=" * 80)


🧬 COMPUTING MODEL-LEVEL nDNA
   Vocabulary indexed: 546 words
   Skipped (no token): 0

✅ nDNA Calculator Initialized:
   Layers: 36
   Vocab size: 151936
   Concreteness words indexed: 546


Computing nDNA:   0%|          | 0/15 [00:00<?, ?it/s]

   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36
   ⚠️ Error: 36

✅ MODEL_RESULTS Computed!

📊 RESULTS SUMMARY:
   Layers:     36 (0 to 35)
   Spectral κ: min=0.000000, max=14.309154
   Thermo Δ:   min=0.105789, max=3.127597
   Belief β:   min=190.913065, max=11169.508594
   nDNA:       min=0.00000000, max=35490.52793333

   🎯 Peak nDNA at Layer 33: 35490.52793333

✅ Cell 5 Complete: MODEL_RESULTS Created
   Variables ready: LAYERS, spectral, thermo, belief, ndna, max_idx


In [15]:
# ============================================================================
# CELL 46: SETUP PLOTLY FOR COLAB DISPLAY
# ============================================================================

print("\n" + "=" * 70)
print("📊 SETTING UP PLOTLY FOR GOOGLE COLAB")
print("=" * 70)

import plotly.io as pio
from IPython.display import display, HTML

# Force Colab renderer
pio.renderers.default = 'colab'

# Alternative: Use notebook renderer
# pio.renderers.default = 'notebook'

# Function to forcefully display plots in Colab
def show_plot(fig, filename=None):
    """
    Forcefully display plotly figure in Google Colab.
    Also saves to HTML file if filename provided.
    """
    # Save if filename provided
    if filename:
        filepath = os.path.join(config.output_dir, filename)
        fig.write_html(filepath, include_plotlyjs='cdn')
        print(f"💾 Saved: {filename}")

    # Method 1: Direct show with colab renderer
    try:
        fig.show(renderer='colab')
    except:
        pass

    # Method 2: Display as HTML (backup)
    try:
        display(HTML(fig.to_html(include_plotlyjs='cdn')))
    except:
        pass

    # Method 3: Use iframe display
    try:
        from IPython.display import IFrame
        import tempfile
        with tempfile.NamedTemporaryFile(suffix='.html', delete=False) as f:
            fig.write_html(f.name, include_plotlyjs='cdn')
            display(IFrame(f.name, width=1000, height=600))
    except:
        pass

# Test display
test_fig = go.Figure()
test_fig.add_trace(go.Scatter(x=[1,2,3], y=[1,2,3], mode='markers+lines', name='Test'))
test_fig.update_layout(title="Test Plot - If you see this, Plotly is working!")

print("\n🧪 Testing Plotly display...")
show_plot(test_fig)
print("✅ If you see the test plot above, display is working!")


📊 SETTING UP PLOTLY FOR GOOGLE COLAB

🧪 Testing Plotly display...


✅ If you see the test plot above, display is working!


In [16]:
# ============================================================================
# CELL 6: ALL 3D AND 2D PLOTS
# ============================================================================

print("\n" + "=" * 80)
print("📊 GENERATING ALL PLOTS")
print("=" * 80)

def save_and_show(fig, filepath):
    """Save figure to HTML and display."""
    fig.write_html(filepath)
    print(f"💾 Saved: {filepath}")
    fig.show()

# ============================================================================
# PLOT 1: 3D nDNA TRAJECTORY
# ============================================================================

print("\n📊 Plot 1: 3D nDNA Trajectory...")

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=LAYERS, y=np.zeros_like(LAYERS), z=ndna,
    mode='lines+markers+text',
    name='nDNA (κ×Δ×β)',
    line=dict(color='#6A0DAD', width=8),
    marker=dict(size=8, color=ndna, colorscale='Plasma',
                colorbar=dict(title="nDNA", x=1.1), showscale=True),
    text=[f"L{int(l)}" for l in LAYERS],
    hovertemplate="<b>Layer %{x:.0f}</b><br>nDNA: %{z:.8f}<extra></extra>"
))

for l, n in zip(LAYERS, ndna):
    fig.add_trace(go.Scatter3d(
        x=[l, l], y=[0, 0], z=[0, n],
        mode='lines', line=dict(color='rgba(106,13,173,0.3)', width=2),
        showlegend=False, hoverinfo='skip'
    ))

fig.add_trace(go.Scatter3d(
    x=[LAYERS[max_idx]], y=[0], z=[ndna[max_idx]],
    mode='markers+text', name=f'Peak (L{int(LAYERS[max_idx])})',
    marker=dict(size=20, color='gold', symbol='diamond'),
    text=[f'MAX: {ndna[max_idx]:.6f}'], textposition='top center'
))

fig.update_layout(
    title=dict(text=f"🧬 nDNA = κ × Δ × β (Layer-wise)<br>"
                    f"<sup>Belief = Mean gradient toward {len(WORD_TO_CONCRETENESS)} words</sup>",
               font=dict(size=18), x=0.5),
    scene=dict(xaxis=dict(title="Layer"), yaxis=dict(title="", showticklabels=False),
               zaxis=dict(title="nDNA"), camera=dict(eye=dict(x=1.8, y=1.5, z=1.0))),
    height=700, width=1000, template='plotly_white'
)
save_and_show(fig, f"{OUTPUT_DIR}/3D_ndna_layerwise.html")

# ============================================================================
# PLOT 2: 3D SPECTRAL (κ)
# ============================================================================

print("\n📊 Plot 2: 3D Spectral Curvature...")

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=LAYERS, y=spectral, z=np.zeros_like(LAYERS),
    mode='lines+markers+text', name='Spectral κ',
    line=dict(color='#E63946', width=6),
    marker=dict(size=6, color=LAYERS, colorscale='Viridis', showscale=True),
    text=[f"L{int(l)}" for l in LAYERS],
    hovertemplate="<b>Layer %{x:.0f}</b><br>κ: %{y:.6f}<extra></extra>"
))
fig.update_layout(
    title=dict(text="🔬 Spectral Curvature (κ)<br><sup>Turning angle on Fisher-Rao sphere</sup>",
               font=dict(size=18), x=0.5),
    scene=dict(xaxis=dict(title="Layer"), yaxis=dict(title="Spectral κ"),
               zaxis=dict(title="", showticklabels=False),
               camera=dict(eye=dict(x=1.8, y=1.5, z=1.0))),
    height=650, width=1000, template='plotly_white'
)
save_and_show(fig, f"{OUTPUT_DIR}/3D_spectral_layerwise.html")

# ============================================================================
# PLOT 3: 3D THERMO (Δ)
# ============================================================================

print("\n📊 Plot 3: 3D Thermodynamic Length...")

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=LAYERS, y=thermo, z=np.zeros_like(LAYERS),
    mode='lines+markers+text', name='Thermo Δ',
    line=dict(color='#2A9D8F', width=6),
    marker=dict(size=6, color=LAYERS, colorscale='Viridis', showscale=True),
    text=[f"L{int(l)}" for l in LAYERS],
    hovertemplate="<b>Layer %{x:.0f}</b><br>Δ: %{y:.6f}<extra></extra>"
))
fig.update_layout(
    title=dict(text="🌡️ Thermodynamic Length (Δ)<br><sup>Fisher-Rao geodesic distance</sup>",
               font=dict(size=18), x=0.5),
    scene=dict(xaxis=dict(title="Layer"), yaxis=dict(title="Thermo Δ"),
               zaxis=dict(title="", showticklabels=False),
               camera=dict(eye=dict(x=1.8, y=1.5, z=1.0))),
    height=650, width=1000, template='plotly_white'
)
save_and_show(fig, f"{OUTPUT_DIR}/3D_thermo_layerwise.html")

# ============================================================================
# PLOT 4: 3D BELIEF (β)
# ============================================================================

print("\n📊 Plot 4: 3D Belief...")

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=LAYERS, y=belief, z=np.zeros_like(LAYERS),
    mode='lines+markers+text', name='Belief β',
    line=dict(color='#457B9D', width=6),
    marker=dict(size=6, color=LAYERS, colorscale='Viridis', showscale=True),
    text=[f"L{int(l)}" for l in LAYERS],
    hovertemplate="<b>Layer %{x:.0f}</b><br>β: %{y:.6f}<extra></extra>"
))
fig.update_layout(
    title=dict(text=f"🧠 Belief (β)<br><sup>Mean gradient toward {len(WORD_TO_CONCRETENESS)} words</sup>",
               font=dict(size=18), x=0.5),
    scene=dict(xaxis=dict(title="Layer"), yaxis=dict(title="Belief β"),
               zaxis=dict(title="", showticklabels=False),
               camera=dict(eye=dict(x=1.8, y=1.5, z=1.0))),
    height=650, width=1000, template='plotly_white'
)
save_and_show(fig, f"{OUTPUT_DIR}/3D_belief_layerwise.html")

# ============================================================================
# PLOT 5: 3D COMPONENT SPACE
# ============================================================================

print("\n📊 Plot 5: 3D Component Space...")

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=spectral, y=thermo, z=belief,
    mode='lines+markers+text', name='Trajectory',
    line=dict(color='#6A0DAD', width=6),
    marker=dict(size=8, color=LAYERS, colorscale='Viridis',
                colorbar=dict(title="Layer", x=1.1), showscale=True),
    text=[f"L{int(l)}" for l in LAYERS],
    hovertemplate="<b>%{text}</b><br>κ:%{x:.4f}<br>Δ:%{y:.4f}<br>β:%{z:.4f}<extra></extra>"
))
fig.add_trace(go.Scatter3d(
    x=[spectral[0]], y=[thermo[0]], z=[belief[0]],
    mode='markers+text', name='Start (L0)',
    marker=dict(size=15, color='#2ECC71', symbol='diamond'),
    text=['L0'], textposition='bottom center'
))
fig.add_trace(go.Scatter3d(
    x=[spectral[-1]], y=[thermo[-1]], z=[belief[-1]],
    mode='markers+text', name=f'End (L{int(LAYERS[-1])})',
    marker=dict(size=15, color='#E74C3C', symbol='square'),
    text=[f'L{int(LAYERS[-1])}'], textposition='top center'
))
fig.update_layout(
    title=dict(text="🧬 3D Component Space: (κ, Δ, β)", font=dict(size=18), x=0.5),
    scene=dict(xaxis=dict(title="Spectral κ"), yaxis=dict(title="Thermo Δ"),
               zaxis=dict(title="Belief β"), camera=dict(eye=dict(x=1.8, y=1.8, z=1.2))),
    height=750, width=1000, template='plotly_white'
)
save_and_show(fig, f"{OUTPUT_DIR}/3D_component_space.html")

# ============================================================================
# PLOT 6: 2D COMBINED METRICS
# ============================================================================

print("\n📊 Plot 6: 2D Combined Metrics...")

fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=['Spectral κ', 'Thermo Δ', 'Belief β', 'nDNA = κ × Δ × β'],
    vertical_spacing=0.08
)

fig.add_trace(go.Scatter(x=LAYERS, y=spectral, mode='lines+markers', name='κ',
    line=dict(color='#E63946', width=3), marker=dict(size=6)), row=1, col=1)
fig.add_trace(go.Scatter(x=LAYERS, y=thermo, mode='lines+markers', name='Δ',
    line=dict(color='#2A9D8F', width=3), marker=dict(size=6)), row=2, col=1)
fig.add_trace(go.Scatter(x=LAYERS, y=belief, mode='lines+markers', name='β',
    line=dict(color='#457B9D', width=3), marker=dict(size=6)), row=3, col=1)
fig.add_trace(go.Scatter(x=LAYERS, y=ndna, mode='lines+markers', name='nDNA',
    line=dict(color='#6A0DAD', width=3), marker=dict(size=6)), row=4, col=1)
fig.add_trace(go.Scatter(x=[LAYERS[max_idx]], y=[ndna[max_idx]], mode='markers',
    marker=dict(size=12, color='gold', symbol='star'), showlegend=False), row=4, col=1)

fig.update_layout(title=dict(text="🧬 nDNA Components", font=dict(size=18)),
                  height=900, template='plotly_white', showlegend=False)
fig.update_xaxes(title_text="Layer", row=4, col=1)
fig.show()
save_and_show(fig, f"{OUTPUT_DIR}/2D_combined_metrics.html")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("✅ ALL PLOTS GENERATED")
print("=" * 80)
print(f"""
📁 Saved to: {OUTPUT_DIR}/
   • 3D_ndna_layerwise.html
   • 3D_spectral_layerwise.html
   • 3D_thermo_layerwise.html
   • 3D_belief_layerwise.html
   • 3D_component_space.html
   • 2D_combined_metrics.html

📈 KEY FINDINGS:
   Peak nDNA:    Layer {int(LAYERS[max_idx])} = {ndna[max_idx]:.8f}
   Max Spectral: Layer {int(LAYERS[np.argmax(spectral)])} = {spectral.max():.6f}
   Max Thermo:   Layer {int(LAYERS[np.argmax(thermo)])} = {thermo.max():.6f}
   Max Belief:   Layer {int(LAYERS[np.argmax(belief)])} = {belief.max():.6f}
""")


📊 GENERATING ALL PLOTS

📊 Plot 1: 3D nDNA Trajectory...
💾 Saved: ./ndna_model_level/3D_ndna_layerwise.html



📊 Plot 2: 3D Spectral Curvature...
💾 Saved: ./ndna_model_level/3D_spectral_layerwise.html



📊 Plot 3: 3D Thermodynamic Length...
💾 Saved: ./ndna_model_level/3D_thermo_layerwise.html



📊 Plot 4: 3D Belief...
💾 Saved: ./ndna_model_level/3D_belief_layerwise.html



📊 Plot 5: 3D Component Space...
💾 Saved: ./ndna_model_level/3D_component_space.html



📊 Plot 6: 2D Combined Metrics...


💾 Saved: ./ndna_model_level/2D_combined_metrics.html



✅ ALL PLOTS GENERATED

📁 Saved to: ./ndna_model_level/
   • 3D_ndna_layerwise.html
   • 3D_spectral_layerwise.html
   • 3D_thermo_layerwise.html
   • 3D_belief_layerwise.html
   • 3D_component_space.html
   • 2D_combined_metrics.html

📈 KEY FINDINGS:
   Peak nDNA:    Layer 33 = 35490.52793333
   Max Spectral: Layer 3 = 14.309154
   Max Thermo:   Layer 35 = 3.127597
   Max Belief:   Layer 35 = 11169.508594



for Llama

In [17]:
# ============================================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================================

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import gc
import os
import re
import time
import requests
from typing import Dict, List, Tuple, Optional, Any
from tqdm.auto import tqdm
from dataclasses import dataclass

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Check for GPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Output directory
OUTPUT_DIR = "./llama3-8B-instruct_ndna"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📁 Output directory: {OUTPUT_DIR}")

print("\n✅ Imports complete")

🖥️ Device: cuda
   GPU: NVIDIA A100 80GB PCIe
   Memory: 85.1 GB
📁 Output directory: ./llama3-8B-instruct_ndna

✅ Imports complete


In [18]:
# ============================================================================
# CELL 2: LOAD MODEL AND TOKENIZER
# ============================================================================

from transformers import AutoModelForCausalLM, AutoTokenizer

# Choose your model (uncomment one)
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_NAME = "Qwen/Qwen2.5-1.5B"
# MODEL_NAME = "meta-llama/Llama-2-7b-hf"
# MODEL_NAME = "gpt2"

print(f"\n{'='*80}")
print(f"🤖 LOADING MODEL: {MODEL_NAME}")
print(f"{'='*80}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print(f"✅ Tokenizer loaded: vocab_size={tokenizer.vocab_size}")

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
    trust_remote_code=True,
    output_hidden_states=True
)

if not torch.cuda.is_available():
    model = model.to(DEVICE)

model.eval()

# Get model info
NUM_LAYERS = model.config.num_hidden_layers
VOCAB_SIZE = model.config.vocab_size
HIDDEN_DIM = model.config.hidden_size

print(f"✅ Model loaded:")
print(f"   Layers: {NUM_LAYERS}")
print(f"   Vocab size: {VOCAB_SIZE}")
print(f"   Hidden dim: {HIDDEN_DIM}")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")


🤖 LOADING MODEL: meta-llama/Llama-3.1-8B-Instruct


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

✅ Tokenizer loaded: vocab_size=128000


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

✅ Model loaded:
   Layers: 32
   Vocab size: 128256
   Hidden dim: 4096
   Parameters: 8030.3M


In [24]:
# ============================================================================
# CELL 3: LOAD 40K BRYSBAERT CONCRETENESS DATA FROM CSV
# ============================================================================

print("\n" + "=" * 80)
print("📚 LOADING BRYSBAERT 40K CONCRETENESS DATASET FROM CSV")
print("=" * 80)

import requests
import pandas as pd
import numpy as np
from typing import Dict
from io import StringIO

def load_brysbaert_40k() -> Dict[str, float]:
    """
    Load Brysbaert et al. (2014) concreteness ratings for ~40K words.
    
    CSV Structure:
    - Column 0: Word (the word itself)
    - Column 1: Bigram
    - Column 2: Conc.M (Concreteness Mean - THIS IS WHAT WE NEED)
    - Column 3: Conc.SD
    - ... other columns
    
    Source: https://github.com/GaurangaKrB/ndna/blob/main/ndna_lib/concreteness/concreteness_scores_original.csv
    """
    
    # GitHub RAW URL for the CSV file
    GITHUB_RAW_URL = "https://raw.githubusercontent.com/GaurangaKrB/ndna/main/ndna_lib/concreteness/concreteness_scores_original.csv"
    
    word_to_concreteness: Dict[str, float] = {}
    
    # =========================================================================
    # METHOD 1: Load from GitHub Raw URL
    # =========================================================================
    try:
        print(f"📥 Downloading from GitHub Raw URL...")
        print(f"   URL: {GITHUB_RAW_URL}")
        
        # Download CSV content
        response = requests.get(GITHUB_RAW_URL, timeout=60)
        response.raise_for_status()
        
        # Read CSV from string content
        csv_content = response.text
        df = pd.read_csv(StringIO(csv_content))
        
        print(f"   ✅ Downloaded successfully!")
        print(f"   Raw shape: {df.shape}")
        print(f"   Columns: {list(df.columns)}")
        
        # The CSV has columns: Word, Bigram, Conc.M, Conc.SD, Unknown, Total, Percent_known, SUBTLEX, Dom_Pos
        # We need: Word (column 0) and Conc.M (column 2, index 2)
        
        # Identify columns
        word_col = df.columns[0]  # 'Word'
        conc_col = df.columns[2]  # 'Conc.M'
        
        print(f"   Word column: '{word_col}'")
        print(f"   Concreteness column: '{conc_col}'")
        
        # Extract word and concreteness
        df['word_clean'] = df[word_col].astype(str).str.lower().str.strip()
        df['concreteness'] = pd.to_numeric(df[conc_col], errors='coerce')
        
        # Filter valid entries
        df_valid = df.dropna(subset=['word_clean', 'concreteness'])
        df_valid = df_valid[df_valid['word_clean'].str.len() > 0]
        df_valid = df_valid[(df_valid['concreteness'] >= 1.0) & (df_valid['concreteness'] <= 5.0)]
        df_valid = df_valid.drop_duplicates(subset=['word_clean'])
        
        # Create dictionary
        word_to_concreteness = dict(zip(df_valid['word_clean'], df_valid['concreteness']))
        
        print(f"\n   ✅ Successfully loaded {len(word_to_concreteness):,} words from GitHub CSV!")
        print(f"   Concreteness range: [{df_valid['concreteness'].min():.2f}, {df_valid['concreteness'].max():.2f}]")
        print(f"   Mean concreteness: {df_valid['concreteness'].mean():.2f}")
        
        # Show sample words
        sorted_words = sorted(word_to_concreteness.items(), key=lambda x: x[1])
        print(f"\n   Sample - Most ABSTRACT (lowest concreteness):")
        for word, score in sorted_words[:5]:
            print(f"      '{word}': {score:.2f}")
        print(f"\n   Sample - Most CONCRETE (highest concreteness):")
        for word, score in sorted_words[-5:]:
            print(f"      '{word}': {score:.2f}")
        
        return word_to_concreteness
        
    except Exception as e:
        print(f"   ⚠️ GitHub download failed: {e}")
        print(f"   Trying alternative methods...")
    
    # =========================================================================
    # METHOD 2: Try loading from local file path
    # =========================================================================
    local_paths = [
        "/Volumes/Research/nDNA_amitava_das/Culture_Work/07Jan2026/concreteness_scores_original.csv",
        "./concreteness_scores_original.csv",
        "../concreteness_scores_original.csv",
        "../../ndna_lib/concreteness/concreteness_scores_original.csv",
        "./ndna_lib/concreteness/concreteness_scores_original.csv",
    ]
    
    for local_path in local_paths:
        try:
            print(f"   Trying local path: {local_path}")
            df = pd.read_csv(local_path)
            
            # Same processing as above
            word_col = df.columns[0]  # 'Word'
            conc_col = df.columns[2]  # 'Conc.M'
            
            df['word_clean'] = df[word_col].astype(str).str.lower().str.strip()
            df['concreteness'] = pd.to_numeric(df[conc_col], errors='coerce')
            
            df_valid = df.dropna(subset=['word_clean', 'concreteness'])
            df_valid = df_valid[df_valid['word_clean'].str.len() > 0]
            df_valid = df_valid[(df_valid['concreteness'] >= 1.0) & (df_valid['concreteness'] <= 5.0)]
            df_valid = df_valid.drop_duplicates(subset=['word_clean'])
            
            word_to_concreteness = dict(zip(df_valid['word_clean'], df_valid['concreteness']))
            
            print(f"   ✅ Loaded {len(word_to_concreteness):,} words from local file!")
            return word_to_concreteness
            
        except Exception as e:
            continue
    
    # =========================================================================
    # METHOD 3: Fallback to comprehensive sample (if all else fails)
    # =========================================================================
    print("\n⚠️ All download methods failed. Using comprehensive fallback sample...")
    print("   (This is a backup - the full 40K dataset is preferred)")
    
    brysbaert_fallback = {
        # HIGH CONCRETENESS (4.5-5.0)
        'sled': 5.00, 'plunger': 4.96, 'human': 4.93, 'waterbed': 4.93, 'cymbal': 4.92,
        'ginger': 4.92, 'bobsled': 4.90, 'cardboard': 4.90, 'olive': 4.90, 'dogsled': 4.89,
        'rubber': 4.86, 'roadsweeper': 4.85, 'soybean': 4.82, 'tangerine': 4.81, 'headrest': 4.80,
        'eucalyptus': 4.77, 'saltwater': 4.77, 'armrest': 4.76, 'paramedic': 4.74, 'liquid': 4.72,
        'billfold': 4.71, 'canine': 4.71, 'flowerbed': 4.71, 'soy': 4.70, 'bald': 4.69,
        'lilac': 4.69, 'hemorrhoid': 4.68, 'orange': 4.66, 'arachnid': 4.65, 'underarm': 4.63,
        'barefoot': 4.62, 'bearded': 4.62, 'thyroid': 4.61, 'wooden': 4.61, 'sleeveless': 4.60,
        'concrete': 4.59, 'panty': 4.59, 'pregnant': 4.59, 'helmeted': 4.58, 'hula': 4.58,
        'female': 4.57, 'crematory': 4.56, 'pigtailed': 4.55, 'traindriver': 4.54, 'sphincter': 4.54,
        'backrest': 4.52, 'blonde': 4.52, 'farmhand': 4.52, 'fat': 4.52, 'hairless': 4.52,
        'afghan': 4.50, 'binocular': 4.50, 'naked': 4.50, 'hairy': 4.48, 'saline': 4.48,
        'sudsy': 4.48, 'wet': 4.46, 'wooded': 4.46, 'tush': 4.45, 'male': 4.45,
        'rusted': 4.44, 'aquamarine': 4.43, 'shirtless': 4.43, 'headless': 4.42, 'solid': 4.42,
        
        # MEDIUM-HIGH (3.5-4.5)
        'hot': 4.31, 'yellow': 4.30, 'dark': 4.29, 'red': 4.24, 'dirty': 4.23,
        'grilled': 4.23, 'sunny': 4.20, 'baked': 4.19, 'sweaty': 4.18, 'bloody': 4.08,
        'dead': 4.07, 'green': 4.07, 'purple': 4.04, 'blind': 4.03, 'frozen': 4.34,
        'military': 4.00, 'sweet': 4.00, 'thick': 4.00, 'pink': 3.93, 'white': 3.89,
        'soft': 3.88, 'chemical': 3.86, 'sharp': 3.86, 'cold': 3.85, 'gray': 3.84,
        'rough': 3.83, 'thin': 3.83, 'dry': 3.77, 'black': 3.76, 'blue': 3.76,
        'hard': 3.76, 'antique': 3.75, 'explosive': 3.74, 'musical': 3.72, 'asleep': 3.71,
        'obese': 3.70, 'elderly': 3.68, 'big': 3.66, 'silent': 3.67, 'little': 3.67,
        'indoor': 3.64, 'wounded': 3.64, 'digital': 3.63, 'creamy': 3.62, 'stiff': 3.62,
        'short': 3.61, 'armed': 3.60, 'full': 3.59, 'messy': 3.59, 'sticky': 3.59,
        
        # MEDIUM (2.5-3.5)
        'electric': 3.56, 'warm': 3.56, 'clear': 3.55, 'huge': 3.54, 'cool': 3.53,
        'alien': 3.52, 'teenage': 3.52, 'drunk': 3.48, 'high': 3.46, 'painful': 3.43,
        'empty': 3.43, 'urban': 3.43, 'active': 3.32, 'healthy': 3.31, 'physical': 3.31,
        'electronic': 3.30, 'deep': 3.38, 'heavy': 3.37, 'large': 3.37, 'slow': 3.28,
        'crowded': 3.28, 'deaf': 3.27, 'single': 3.27, 'small': 3.22, 'ugly': 3.23,
        'bare': 3.21, 'massive': 3.21, 'close': 3.20, 'raw': 3.35, 'low': 3.34,
        'shiny': 3.33, 'whole': 3.25, 'separate': 3.25, 'commercial': 3.24, 'safe': 3.41,
        'criminal': 3.48, 'communist': 3.50, 'identical': 3.50, 'junior': 3.50, 'vacant': 3.50,
        
        # LOW (1.5-2.5) - Abstract
        'daily': 2.97, 'endless': 2.52, 'sudden': 2.48, 'random': 2.45, 'final': 2.41,
        'constant': 2.38, 'basic': 2.34, 'common': 2.31, 'normal': 2.28, 'general': 2.24,
        'simple': 2.48, 'complex': 2.17, 'special': 2.14, 'major': 2.10, 'main': 2.07,
        'real': 2.03, 'true': 1.97, 'free': 2.45, 'open': 2.90, 'wrong': 1.90,
        'right': 2.17, 'good': 1.87, 'bad': 1.93, 'new': 2.27, 'old': 2.83,
        'great': 1.80, 'best': 1.77, 'important': 1.73, 'different': 1.70, 'same': 1.67,
        'other': 1.63, 'next': 1.97, 'last': 2.13, 'first': 1.90, 'only': 1.53,
        'own': 1.57, 'such': 1.43, 'able': 1.47, 'sure': 1.50, 'likely': 1.40,
        
        # VERY LOW (1.0-1.5) - Highly abstract
        'pushiness': 2.48, 'underdevelop': 2.37, 'tirelessness': 2.28, 'oldfashioned': 2.26,
        'wellmannered': 2.25, 'dismissiveness': 1.83, 'spitefulness': 1.80, 'untruthfulness': 1.73,
        'dispiritedness': 1.56, 'hoover': 3.76, 'shopkeeping': 3.18, 'hairdress': 3.93,
        'pharmaceutics': 3.77, 'essentialness': 1.04,
    }
    
    return brysbaert_fallback


# ============================================================================
# EXECUTE: Load the concreteness data
# ============================================================================

WORD_TO_CONCRETENESS = load_brysbaert_40k()

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================

print(f"\n" + "=" * 80)
print("📊 CONCRETENESS DATASET SUMMARY")
print("=" * 80)

conc_values = list(WORD_TO_CONCRETENESS.values())

print(f"   Total words loaded: {len(WORD_TO_CONCRETENESS):,}")
print(f"   Concreteness range: [{min(conc_values):.2f}, {max(conc_values):.2f}]")
print(f"   Mean concreteness:  {np.mean(conc_values):.2f}")
print(f"   Std concreteness:   {np.std(conc_values):.2f}")
print(f"   Median:             {np.median(conc_values):.2f}")

# Distribution by category
abstract = sum(1 for v in conc_values if v < 2.5)
medium = sum(1 for v in conc_values if 2.5 <= v < 3.5)
concrete = sum(1 for v in conc_values if v >= 3.5)

print(f"\n   Distribution by category:")
print(f"   ┌─────────────────────────────────────────────────┐")
print(f"   │ Abstract  (1.0-2.5): {abstract:>6,} ({100*abstract/len(conc_values):>5.1f}%) │")
print(f"   │ Medium    (2.5-3.5): {medium:>6,} ({100*medium/len(conc_values):>5.1f}%) │")
print(f"   │ Concrete  (3.5-5.0): {concrete:>6,} ({100*concrete/len(conc_values):>5.1f}%) │")
print(f"   └─────────────────────────────────────────────────┘")

# Verify we have enough words for meaningful belief calculation
if len(WORD_TO_CONCRETENESS) >= 1000:
    print(f"\n   ✅ Sufficient vocabulary for belief calculation!")
else:
    print(f"\n   ⚠️ Limited vocabulary ({len(WORD_TO_CONCRETENESS)} words) - results may be less reliable")

print("=" * 80)


📚 LOADING BRYSBAERT 40K CONCRETENESS DATASET FROM CSV
📥 Downloading from GitHub Raw URL...
   URL: https://raw.githubusercontent.com/GaurangaKrB/ndna/main/ndna_lib/concreteness/concreteness_scores_original.csv
   ⚠️ GitHub download failed: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/GaurangaKrB/ndna/main/ndna_lib/concreteness/concreteness_scores_original.csv
   Trying alternative methods...
   Trying local path: /Volumes/Research/nDNA_amitava_das/Culture_Work/07Jan2026/concreteness_scores_original.csv
   Trying local path: ./concreteness_scores_original.csv
   ✅ Loaded 39,954 words from local file!

📊 CONCRETENESS DATASET SUMMARY
   Total words loaded: 39,954
   Concreteness range: [1.04, 5.00]
   Mean concreteness:  3.04
   Std concreteness:   1.04
   Median:             2.88

   Distribution by category:
   ┌─────────────────────────────────────────────────┐
   │ Abstract  (1.0-2.5): 15,447 ( 38.7%) │
   │ Medium    (2.5-3.5): 10,913 ( 27.3%) │
   │ Concre

In [25]:
# ============================================================================
# CELL 4: COMPUTE MODEL-LEVEL nDNA (SPECTRAL, THERMO, BELIEF)
# ============================================================================
# nDNA = Spectral(κ) × Thermo(Δ) × Belief(β)
#
# CORRECT DEFINITIONS:
# - Spectral (κ): Turning-angle curvature on Fisher-Rao sphere
# - Thermo (Δ): Fisher-Rao geodesic distance between consecutive layers
# - Belief (β): Mean gradient magnitude toward ALL vocabulary words (40K)
# ============================================================================

import torch
import torch.nn.functional as F
import numpy as np
from typing import Dict, List, Tuple, Any
from tqdm.auto import tqdm
import gc

print("=" * 80)
print("🧬 COMPUTING MODEL-LEVEL nDNA")
print("   nDNA = Spectral(κ) × Thermo(Δ) × Belief(β)")
print("   Using", len(WORD_TO_CONCRETENESS), "vocabulary words for belief calculation")
print("=" * 80)


class ModelNDNACalculator:
    """
    Compute nDNA components for a language model.
    
    nDNA = κ × Δ × β
    
    Components:
    -----------
    1. SPECTRAL CURVATURE (κ): Turning angle on Fisher-Rao sphere
    2. THERMODYNAMIC LENGTH (Δ): Fisher-Rao geodesic per layer
    3. BELIEF (β): Mean gradient toward ALL vocabulary words
    """
    
    def __init__(
        self,
        model,
        tokenizer,
        word_to_concreteness: Dict[str, float],
        device: torch.device,
        eps: float = 1e-9
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.eps = eps
        self.word_to_concreteness = word_to_concreteness
        
        # Get model config
        self.num_layers = model.config.num_hidden_layers
        self.vocab_size = model.config.vocab_size
        
        # Get LM head
        self.lm_head = self._get_lm_head()
        
        # Build vocabulary index
        self._build_vocabulary_index()
        
        print(f"\n✅ nDNA Calculator initialized:")
        print(f"   Model layers: {self.num_layers}")
        print(f"   Vocab size: {self.vocab_size:,}")
        print(f"   Words indexed for belief: {len(self.valid_token_ids):,}")
    
    def _get_lm_head(self):
        """Extract LM head from model."""
        if hasattr(self.model, 'lm_head'):
            return self.model.lm_head
        elif hasattr(self.model, 'model') and hasattr(self.model.model, 'lm_head'):
            return self.model.model.lm_head
        elif hasattr(self.model, 'cls'):
            return self.model.cls
        else:
            # Try to find it
            for name, module in self.model.named_modules():
                if 'lm_head' in name.lower():
                    return module
            raise ValueError("Cannot find lm_head in model")
    
    def _build_vocabulary_index(self):
        """Build mapping: token_id → concreteness score for all 40K words."""
        self.token_id_to_concreteness: Dict[int, float] = {}
        self.token_id_to_word: Dict[int, str] = {}
        self.valid_token_ids: List[int] = []
        self.valid_concreteness: List[float] = []
        
        indexed = 0
        skipped = 0
        
        for word, conc_score in self.word_to_concreteness.items():
            # Try with space prefix (standard for most tokenizers)
            tokens = self.tokenizer.encode(f" {word}", add_special_tokens=False)
            if len(tokens) == 0:
                tokens = self.tokenizer.encode(word, add_special_tokens=False)
            
            if len(tokens) > 0:
                token_id = tokens[0]  # First token
                
                # Avoid duplicates and special tokens
                if token_id not in self.token_id_to_concreteness and token_id < self.vocab_size:
                    self.token_id_to_concreteness[token_id] = conc_score
                    self.token_id_to_word[token_id] = word
                    self.valid_token_ids.append(token_id)
                    self.valid_concreteness.append(conc_score)
                    indexed += 1
            else:
                skipped += 1
        
        # Convert to tensors
        self.valid_token_ids_tensor = torch.tensor(
            self.valid_token_ids, dtype=torch.long, device=self.device
        )
        self.valid_concreteness_tensor = torch.tensor(
            self.valid_concreteness, dtype=torch.float32, device=self.device
        )
        
        print(f"   Indexed: {indexed:,} words")
        print(f"   Skipped: {skipped:,} words (no valid token)")
        if len(self.valid_concreteness) > 0:
            print(f"   Concreteness range: [{min(self.valid_concreteness):.2f}, {max(self.valid_concreteness):.2f}]")
    
    # =========================================================================
    # FISHER-RAO GEOMETRY
    # =========================================================================
    
    def _safe_arccos(self, x: torch.Tensor) -> torch.Tensor:
        """Numerically stable arccos."""
        return torch.arccos(torch.clamp(x, -1.0 + self.eps, 1.0 - self.eps))
    
    def _fisher_rao_embed(self, probs: torch.Tensor) -> torch.Tensor:
        """
        Embed probability distribution onto Fisher-Rao unit sphere.
        u = sqrt(p) / ||sqrt(p)||
        """
        probs = torch.clamp(probs, min=self.eps)
        sqrt_p = torch.sqrt(probs)
        norm = torch.norm(sqrt_p, dim=-1, keepdim=True)
        return sqrt_p / (norm + self.eps)
    
    # =========================================================================
    # SPECTRAL CURVATURE (κ)
    # =========================================================================
    
    def compute_spectral_curvature(
        self,
        u_prev: torch.Tensor,
        u_curr: torch.Tensor,
        u_next: torch.Tensor
    ) -> float:
        """
        Compute turning-angle curvature at u_curr.
        κ = θ / (a + b)
        """
        if u_prev.dim() > 1:
            u_prev = u_prev.mean(dim=0)
            u_curr = u_curr.mean(dim=0)
            u_next = u_next.mean(dim=0)
        
        # Arc lengths
        dot_prev_curr = torch.sum(u_prev * u_curr)
        dot_curr_next = torch.sum(u_curr * u_next)
        dot_prev_next = torch.sum(u_prev * u_next)
        
        a = self._safe_arccos(dot_prev_curr)
        b = self._safe_arccos(dot_curr_next)
        c = self._safe_arccos(dot_prev_next)
        
        # Turning angle via spherical law of cosines
        sin_a = torch.sin(a)
        sin_b = torch.sin(b)
        denom = sin_a * sin_b
        
        if float(denom.cpu()) < self.eps:
            return 0.0
        
        cos_theta = (torch.cos(c) - torch.cos(a) * torch.cos(b)) / denom
        cos_theta = torch.clamp(cos_theta, -1.0 + self.eps, 1.0 - self.eps)
        theta = torch.arccos(cos_theta)
        
        kappa = theta / (a + b + self.eps)
        return float(kappa.cpu())
    
    # =========================================================================
    # THERMODYNAMIC LENGTH (Δ)
    # =========================================================================
    
    def compute_thermo_length(
        self,
        u_curr: torch.Tensor,
        u_next: torch.Tensor
    ) -> float:
        """
        Compute Fisher-Rao geodesic distance.
        Δ = 2 × arccos(<u_curr, u_next>)
        """
        if u_curr.dim() > 1:
            u_curr = u_curr.mean(dim=0)
            u_next = u_next.mean(dim=0)
        
        dot = torch.sum(u_curr * u_next)
        angle = self._safe_arccos(dot)
        delta = 2.0 * angle
        return float(delta.cpu())
    
    # =========================================================================
    # BELIEF (β) - Mean gradient toward ALL vocabulary words
    # =========================================================================
    
    def compute_belief_all_words(
        self,
        probs: torch.Tensor,
        u: torch.Tensor
    ) -> Tuple[float, float, np.ndarray]:
        """
        Compute belief toward ALL words in vocabulary.
        
        Returns:
            - mean_belief: Average belief across all words
            - belief_conc_corr: Correlation(belief, concreteness)
            - per_word_beliefs: Array of beliefs
        """
        if probs.dim() > 1:
            probs = probs.mean(dim=0)
            u = u.mean(dim=0)
        
        n_words = len(self.valid_token_ids)
        if n_words == 0:
            return 0.0, 0.0, np.array([])
        
        beliefs = torch.zeros(n_words, device=self.device)
        sqrt_p = torch.sqrt(probs + self.eps)
        
        # Compute belief for each word
        for i, token_id in enumerate(self.valid_token_ids):
            # One-hot for this token
            one_hot = torch.zeros_like(probs)
            one_hot[token_id] = 1.0
            
            # Gradient toward this word
            g = one_hot - probs
            
            # Natural gradient (Fisher-Rao)
            t = 0.5 * g / sqrt_p
            
            # Project onto tangent space
            u_dot_t = torch.sum(t * u)
            t_tangent = t - u_dot_t * u
            
            # Belief magnitude
            beliefs[i] = torch.norm(t_tangent)
        
        # Mean belief
        mean_belief = float(beliefs.mean().cpu())
        
        # Correlation with concreteness
        beliefs_np = beliefs.cpu().numpy()
        conc_np = self.valid_concreteness_tensor.cpu().numpy()
        
        if len(beliefs_np) > 2 and np.std(beliefs_np) > 0 and np.std(conc_np) > 0:
            corr = np.corrcoef(beliefs_np, conc_np)[0, 1]
            if np.isnan(corr):
                corr = 0.0
        else:
            corr = 0.0
        
        return mean_belief, corr, beliefs_np
    
    # =========================================================================
    # MAIN nDNA COMPUTATION
    # =========================================================================
    
    @torch.no_grad()
    def compute_ndna(
        self,
        prompts: List[str],
        batch_size: int = 1
    ) -> Dict[str, Any]:
        """
        Compute MODEL-LEVEL nDNA across prompts.
        
        Returns dict with arrays:
            - layers: [0, 1, 2, ..., num_layers]
            - spectral: spectral curvature per layer
            - thermo: thermodynamic length per layer
            - belief: mean belief per layer
            - belief_corr: belief-concreteness correlation per layer
            - ndna: nDNA = κ × Δ × β
        """
        n_layers = self.num_layers + 1  # +1 for embedding layer
        
        # Accumulators for each layer
        all_spectral = {l: [] for l in range(n_layers)}
        all_thermo = {l: [] for l in range(n_layers)}
        all_belief = {l: [] for l in range(n_layers)}
        all_belief_corr = {l: [] for l in range(n_layers)}
        
        print(f"\n🔄 Processing {len(prompts)} prompts...")
        
        for prompt in tqdm(prompts, desc="Computing nDNA"):
            try:
                # Tokenize
                inputs = self.tokenizer(
                    prompt,
                    return_tensors="pt",
                    truncation=True,
                    max_length=128
                )
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                
                # Forward pass with hidden states
                outputs = self.model(
                    **inputs,
                    output_hidden_states=True,
                    return_dict=True
                )
                
                hidden_states = outputs.hidden_states
                actual_layers = len(hidden_states)
                
                # Compute probs and embeddings at each layer
                probs_list = []
                u_list = []
                
                for layer_idx in range(actual_layers):
                    h = hidden_states[layer_idx].squeeze(0)  # [T, D]
                    
                    # Project through lm_head
                    logits = self.lm_head(h.to(self.lm_head.weight.dtype))
                    probs = F.softmax(logits.float(), dim=-1).mean(dim=0)  # [V]
                    probs = torch.clamp(probs, min=self.eps)
                    
                    # Fisher-Rao embedding
                    u = self._fisher_rao_embed(probs)
                    
                    probs_list.append(probs)
                    u_list.append(u)
                
                # Compute metrics at each layer
                for layer_idx in range(actual_layers):
                    # SPECTRAL CURVATURE (needs 3 consecutive layers)
                    if 1 <= layer_idx < actual_layers - 1:
                        kappa = self.compute_spectral_curvature(
                            u_list[layer_idx - 1],
                            u_list[layer_idx],
                            u_list[layer_idx + 1]
                        )
                        all_spectral[layer_idx].append(kappa)
                    
                    # THERMODYNAMIC LENGTH (needs 2 consecutive layers)
                    if layer_idx < actual_layers - 1:
                        delta = self.compute_thermo_length(
                            u_list[layer_idx],
                            u_list[layer_idx + 1]
                        )
                        all_thermo[layer_idx].append(delta)
                    
                    # BELIEF (can compute at any layer)
                    beta_mean, beta_corr, _ = self.compute_belief_all_words(
                        probs_list[layer_idx],
                        u_list[layer_idx]
                    )
                    all_belief[layer_idx].append(beta_mean)
                    all_belief_corr[layer_idx].append(beta_corr)
                
            except Exception as e:
                print(f"   ⚠️ Error: {e}")
                continue
            
            # Memory cleanup
            if self.device.type == 'cuda':
                torch.cuda.empty_cache()
            gc.collect()
        
        # Aggregate: mean across prompts
        layers = np.array(list(range(n_layers)))
        spectral = np.array([np.mean(all_spectral[l]) if all_spectral[l] else 0.0 for l in range(n_layers)])
        thermo = np.array([np.mean(all_thermo[l]) if all_thermo[l] else 0.0 for l in range(n_layers)])
        belief = np.array([np.mean(all_belief[l]) if all_belief[l] else 0.0 for l in range(n_layers)])
        belief_corr = np.array([np.mean(all_belief_corr[l]) if all_belief_corr[l] else 0.0 for l in range(n_layers)])
        
        # nDNA = κ × Δ × β
        ndna = spectral * thermo * belief
        
        return {
            'layers': layers,
            'spectral': spectral,
            'thermo': thermo,
            'belief': belief,
            'belief_corr': belief_corr,
            'ndna': ndna
        }


# ============================================================================
# EXECUTE: Compute nDNA
# ============================================================================

# Neutral prompts for model-level analysis
ANALYSIS_PROMPTS = [
    "The concept is",
    "This represents",
    "It means",
    "The idea of",
    "One could say that",
    "In essence,",
    "The nature of",
    "What we understand as",
    "The definition involves",
    "Simply put,",
    "The meaning relates to",
    "This can be described as",
    "In other words,",
    "The significance of",
    "What matters here is",
]

# Initialize calculator
calculator = ModelNDNACalculator(
    model=model,
    tokenizer=tokenizer,
    word_to_concreteness=WORD_TO_CONCRETENESS,
    device=DEVICE
)

# Compute nDNA
MODEL_RESULTS = calculator.compute_ndna(ANALYSIS_PROMPTS)

# ============================================================================
# EXTRACT VARIABLES FOR PLOTTING
# ============================================================================

LAYERS = MODEL_RESULTS['layers']
spectral = MODEL_RESULTS['spectral']
thermo = MODEL_RESULTS['thermo']
belief = MODEL_RESULTS['belief']
belief_corr = MODEL_RESULTS['belief_corr']
ndna = MODEL_RESULTS['ndna']

# ============================================================================
# RESULTS SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("✅ nDNA COMPUTATION COMPLETE")
print("=" * 80)

print(f"""
┌─────────────────────────────────────────────────────────────────────────────┐
│                        nDNA RESULTS SUMMARY                                 │
├─────────────────────────────────────────────────────────────────────────────┤
│  Vocabulary words used: {len(WORD_TO_CONCRETENESS):>10,}                                        │
│  Number of layers:      {len(LAYERS):>10}                                        │
│  Prompts analyzed:      {len(ANALYSIS_PROMPTS):>10}                                        │
├─────────────────────────────────────────────────────────────────────────────┤
│  SPECTRAL κ (Curvature):                                                    │
│    Range: [{spectral.min():.6f}, {spectral.max():.6f}]                              │
│    Peak at Layer {int(LAYERS[np.argmax(spectral)])}: {spectral.max():.6f}                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│  THERMO Δ (Geodesic Length):                                                │
│    Range: [{thermo.min():.6f}, {thermo.max():.6f}]                              │
│    Peak at Layer {int(LAYERS[np.argmax(thermo)])}: {thermo.max():.6f}                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│  BELIEF β (Mean Gradient):                                                  │
│    Range: [{belief.min():.6f}, {belief.max():.6f}]                              │
│    Peak at Layer {int(LAYERS[np.argmax(belief)])}: {belief.max():.6f}                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│  nDNA = κ × Δ × β:                                                          │
│    Range: [{ndna.min():.8f}, {ndna.max():.8f}]                          │
│    Peak at Layer {int(LAYERS[np.argmax(ndna)])}: {ndna.max():.8f}                                │
└─────────────────────────────────────────────────────────────────────────────┘
""")

# Layer-by-layer breakdown
print("\n📊 Layer-by-Layer Breakdown:")
print("-" * 75)
print(f"{'Layer':>6} | {'Spectral κ':>12} | {'Thermo Δ':>12} | {'Belief β':>12} | {'nDNA':>14}")
print("-" * 75)
for i, layer in enumerate(LAYERS):
    print(f"{int(layer):>6} | {spectral[i]:>12.6f} | {thermo[i]:>12.6f} | {belief[i]:>12.6f} | {ndna[i]:>14.8f}")
print("-" * 75)

print("\n✅ MODEL_RESULTS dictionary is now available for plotting!")
print("   Keys:", list(MODEL_RESULTS.keys()))

🧬 COMPUTING MODEL-LEVEL nDNA
   nDNA = Spectral(κ) × Thermo(Δ) × Belief(β)
   Using 39954 vocabulary words for belief calculation
   Indexed: 14,367 words
   Skipped: 0 words (no valid token)
   Concreteness range: [1.04, 5.00]

✅ nDNA Calculator initialized:
   Model layers: 32
   Vocab size: 128,256
   Words indexed for belief: 14,367

🔄 Processing 15 prompts...


Computing nDNA:   0%|          | 0/15 [00:00<?, ?it/s]


✅ nDNA COMPUTATION COMPLETE

┌─────────────────────────────────────────────────────────────────────────────┐
│                        nDNA RESULTS SUMMARY                                 │
├─────────────────────────────────────────────────────────────────────────────┤
│  Vocabulary words used:     39,954                                        │
│  Number of layers:              33                                        │
│  Prompts analyzed:              15                                        │
├─────────────────────────────────────────────────────────────────────────────┤
│  SPECTRAL κ (Curvature):                                                    │
│    Range: [0.000000, 126.362116]                              │
│    Peak at Layer 3: 126.362116                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│  THERMO Δ (Geodesic Length):                                                │
│    Range: [0.000000, 2.001260]        

In [29]:
# ============================================================================
# CELL 5: COMPLETE 3D AND 2D VISUALIZATION SUITE FOR nDNA
# ============================================================================
# All plots: Spectral(κ), Thermo(Δ), Belief(β), nDNA = κ × Δ × β
# Layer-by-layer analysis with interactive 3D and publication-ready 2D plots
# ============================================================================

import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# ============================================================================
# SETUP
# ============================================================================

OUTPUT_DIR = "./ndna_visualizations"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("=" * 80)
print("📊 GENERATING COMPLETE nDNA VISUALIZATION SUITE")
print("=" * 80)

# ============================================================================
# EXTRACT AND VALIDATE DATA FROM MODEL_RESULTS
# ============================================================================

print("\n📥 Extracting data from MODEL_RESULTS...")

# Extract arrays
LAYERS = np.array(MODEL_RESULTS['layers'], dtype=np.float64)
spectral = np.array(MODEL_RESULTS['spectral'], dtype=np.float64)
thermo = np.array(MODEL_RESULTS['thermo'], dtype=np.float64)
belief = np.array(MODEL_RESULTS['belief'], dtype=np.float64)
belief_corr = np.array(MODEL_RESULTS['belief_corr'], dtype=np.float64)
ndna = np.array(MODEL_RESULTS['ndna'], dtype=np.float64)

# Validate
print(f"   ✅ LAYERS:      shape={LAYERS.shape}, range=[{int(LAYERS.min())}, {int(LAYERS.max())}]")
print(f"   ✅ spectral(κ): shape={spectral.shape}, range=[{spectral.min():.6f}, {spectral.max():.6f}]")
print(f"   ✅ thermo(Δ):   shape={thermo.shape}, range=[{thermo.min():.6f}, {thermo.max():.6f}]")
print(f"   ✅ belief(β):   shape={belief.shape}, range=[{belief.min():.6f}, {belief.max():.6f}]")
print(f"   ✅ nDNA:        shape={ndna.shape}, range=[{ndna.min():.8f}, {ndna.max():.8f}]")

# Key indices
peak_ndna_idx = int(np.argmax(ndna))
peak_spectral_idx = int(np.argmax(spectral))
peak_thermo_idx = int(np.argmax(thermo))
peak_belief_idx = int(np.argmax(belief))

print(f"\n   📍 Peak Locations:")
print(f"      nDNA peak:     Layer {int(LAYERS[peak_ndna_idx])}")
print(f"      Spectral peak: Layer {int(LAYERS[peak_spectral_idx])}")
print(f"      Thermo peak:   Layer {int(LAYERS[peak_thermo_idx])}")
print(f"      Belief peak:   Layer {int(LAYERS[peak_belief_idx])}")

# ============================================================================
# COLOR SCHEMES
# ============================================================================

COLORS = {
    'spectral': '#E63946',      # Red
    'thermo': '#2A9D8F',        # Teal
    'belief': '#457B9D',        # Blue
    'ndna': '#6A0DAD',          # Purple
    'start': '#2ECC71',         # Green
    'end': '#E74C3C',           # Red
    'peak': '#FFD700',          # Gold
    'grid': 'rgba(128,128,128,0.2)'
}

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def save_and_show(fig, filename, show=True):
    """Save figure to HTML and optionally display."""
    filepath = f"{OUTPUT_DIR}/{filename}"
    fig.write_html(filepath)
    print(f"   💾 Saved: {filepath}")
    if show:
        fig.show()
    return filepath

def create_layer_labels(layers):
    """Create layer labels like 'L0', 'L1', etc."""
    return [f"L{int(l)}" for l in layers]

# ============================================================================
# 3D PLOT 1: nDNA TRAJECTORY (Primary Visualization)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 1: 3D nDNA TRAJECTORY (Layer-wise)")
print("=" * 80)

fig = go.Figure()

# Main nDNA trajectory line
fig.add_trace(go.Scatter3d(
    x=LAYERS,
    y=np.zeros_like(LAYERS),
    z=ndna,
    mode='lines+markers',
    name='nDNA = κ × Δ × β',
    line=dict(color=COLORS['ndna'], width=10),
    marker=dict(
        size=8,
        color=ndna,
        colorscale='Plasma',
        colorbar=dict(
            title=dict(text="nDNA", font=dict(size=14)),
            x=1.02,
            len=0.8
        ),
        showscale=True
    ),
    text=create_layer_labels(LAYERS),
    hovertemplate="<b>%{text}</b><br>nDNA: %{z:.8f}<extra></extra>"
))

# Vertical fence lines (stems)
for i, (layer, n) in enumerate(zip(LAYERS, ndna)):
    fig.add_trace(go.Scatter3d(
        x=[layer, layer],
        y=[0, 0],
        z=[0, n],
        mode='lines',
        line=dict(color='rgba(106, 13, 173, 0.4)', width=3),
        showlegend=False,
        hoverinfo='skip'
    ))

# Peak marker
fig.add_trace(go.Scatter3d(
    x=[LAYERS[peak_ndna_idx]],
    y=[0],
    z=[ndna[peak_ndna_idx]],
    mode='markers+text',
    name=f'Peak (L{int(LAYERS[peak_ndna_idx])})',
    marker=dict(size=18, color=COLORS['peak'], symbol='diamond'),
    text=[f'PEAK: {ndna[peak_ndna_idx]:.6f}'],
    textposition='top center',
    textfont=dict(size=12, color='black'),
    hovertemplate="<b>PEAK nDNA</b><br>Layer: %{x:.0f}<br>nDNA: %{z:.8f}<extra></extra>"
))

# Start marker
fig.add_trace(go.Scatter3d(
    x=[LAYERS[0]],
    y=[0],
    z=[ndna[0]],
    mode='markers+text',
    name='Start (L0)',
    marker=dict(size=12, color=COLORS['start'], symbol='circle'),
    text=['START'],
    textposition='bottom center',
    textfont=dict(size=10),
    hovertemplate="<b>START</b><br>Layer: 0<br>nDNA: %{z:.8f}<extra></extra>"
))

# End marker
fig.add_trace(go.Scatter3d(
    x=[LAYERS[-1]],
    y=[0],
    z=[ndna[-1]],
    mode='markers+text',
    name=f'End (L{int(LAYERS[-1])})',
    marker=dict(size=12, color=COLORS['end'], symbol='square'),
    text=['END'],
    textposition='top center',
    textfont=dict(size=10),
    hovertemplate="<b>END</b><br>Layer: %{x:.0f}<br>nDNA: %{z:.8f}<extra></extra>"
))

fig.update_layout(
    title=dict(
        text=f"🧬 nDNA = κ × Δ × β (Layer-wise Trajectory)<br>"
             f"<sup>Model: {len(LAYERS)} layers | Vocabulary: {len(WORD_TO_CONCRETENESS):,} words | Peak at Layer {int(LAYERS[peak_ndna_idx])}</sup>",
        font=dict(size=20),
        x=0.5
    ),
    scene=dict(
        xaxis=dict(title="Layer", tickmode='linear', dtick=max(1, len(LAYERS)//10)),
        yaxis=dict(title="", showticklabels=False, showgrid=False),
        zaxis=dict(title="nDNA (κ × Δ × β)"),
        camera=dict(eye=dict(x=1.8, y=1.5, z=1.0)),
        aspectratio=dict(x=2, y=0.5, z=1)
    ),
    height=750,
    width=1100,
    template='plotly_white',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

save_and_show(fig, "01_3D_ndna_trajectory.html")


# ============================================================================
# 3D PLOT 2: SPECTRAL CURVATURE (κ)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 2: 3D SPECTRAL CURVATURE (κ)")
print("=" * 80)

fig = go.Figure()

# Main spectral trajectory
fig.add_trace(go.Scatter3d(
    x=LAYERS,
    y=np.zeros_like(LAYERS),
    z=spectral,
    mode='lines+markers',
    name='Spectral κ',
    line=dict(color=COLORS['spectral'], width=8),
    marker=dict(
        size=7,
        color=spectral,
        colorscale='Reds',
        colorbar=dict(
            title=dict(text="κ", font=dict(size=14)),
            x=1.02,
            len=0.8
        ),
        showscale=True
    ),
    text=create_layer_labels(LAYERS),
    hovertemplate="<b>%{text}</b><br>Spectral κ: %{z:.6f}<extra></extra>"
))

# Fence lines
for layer, s in zip(LAYERS, spectral):
    fig.add_trace(go.Scatter3d(
        x=[layer, layer],
        y=[0, 0],
        z=[0, s],
        mode='lines',
        line=dict(color='rgba(230, 57, 70, 0.3)', width=2),
        showlegend=False,
        hoverinfo='skip'
    ))

# Peak marker
fig.add_trace(go.Scatter3d(
    x=[LAYERS[peak_spectral_idx]],
    y=[0],
    z=[spectral[peak_spectral_idx]],
    mode='markers+text',
    name=f'Peak (L{int(LAYERS[peak_spectral_idx])})',
    marker=dict(size=16, color=COLORS['peak'], symbol='diamond'),
    text=[f'PEAK: {spectral[peak_spectral_idx]:.4f}'],
    textposition='top center',
    textfont=dict(size=11, color='black')
))

fig.update_layout(
    title=dict(
        text="🔬 Spectral Curvature (κ) - Turning Angle on Fisher-Rao Sphere<br>"
             "<sup>κ = θ / (a + b) where θ is spherical turning angle between consecutive layers</sup>",
        font=dict(size=18),
        x=0.5
    ),
    scene=dict(
        xaxis=dict(title="Layer", tickmode='linear', dtick=max(1, len(LAYERS)//10)),
        yaxis=dict(title="", showticklabels=False, showgrid=False),
        zaxis=dict(title="Spectral κ"),
        camera=dict(eye=dict(x=1.8, y=1.5, z=1.0)),
        aspectratio=dict(x=2, y=0.5, z=1)
    ),
    height=700,
    width=1050,
    template='plotly_white',
    legend=dict(x=0.02, y=0.98)
)

save_and_show(fig, "02_3D_spectral_curvature.html")


# ============================================================================
# 3D PLOT 3: THERMODYNAMIC LENGTH (Δ)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 3: 3D THERMODYNAMIC LENGTH (Δ)")
print("=" * 80)

fig = go.Figure()

# Main thermo trajectory
fig.add_trace(go.Scatter3d(
    x=LAYERS,
    y=np.zeros_like(LAYERS),
    z=thermo,
    mode='lines+markers',
    name='Thermo Δ',
    line=dict(color=COLORS['thermo'], width=8),
    marker=dict(
        size=7,
        color=thermo,
        colorscale='Teal',
        colorbar=dict(
            title=dict(text="Δ", font=dict(size=14)),
            x=1.02,
            len=0.8
        ),
        showscale=True
    ),
    text=create_layer_labels(LAYERS),
    hovertemplate="<b>%{text}</b><br>Thermo Δ: %{z:.6f}<extra></extra>"
))

# Fence lines
for layer, t in zip(LAYERS, thermo):
    fig.add_trace(go.Scatter3d(
        x=[layer, layer],
        y=[0, 0],
        z=[0, t],
        mode='lines',
        line=dict(color='rgba(42, 157, 143, 0.3)', width=2),
        showlegend=False,
        hoverinfo='skip'
    ))

# Peak marker
fig.add_trace(go.Scatter3d(
    x=[LAYERS[peak_thermo_idx]],
    y=[0],
    z=[thermo[peak_thermo_idx]],
    mode='markers+text',
    name=f'Peak (L{int(LAYERS[peak_thermo_idx])})',
    marker=dict(size=16, color=COLORS['peak'], symbol='diamond'),
    text=[f'PEAK: {thermo[peak_thermo_idx]:.4f}'],
    textposition='top center',
    textfont=dict(size=11, color='black')
))

fig.update_layout(
    title=dict(
        text="🌡️ Thermodynamic Length (Δ) - Fisher-Rao Geodesic Distance<br>"
             "<sup>Δ = 2 × arccos(⟨u_ℓ, u_{ℓ+1}⟩) measuring information geometry path length</sup>",
        font=dict(size=18),
        x=0.5
    ),
    scene=dict(
        xaxis=dict(title="Layer", tickmode='linear', dtick=max(1, len(LAYERS)//10)),
        yaxis=dict(title="", showticklabels=False, showgrid=False),
        zaxis=dict(title="Thermo Δ"),
        camera=dict(eye=dict(x=1.8, y=1.5, z=1.0)),
        aspectratio=dict(x=2, y=0.5, z=1)
    ),
    height=700,
    width=1050,
    template='plotly_white',
    legend=dict(x=0.02, y=0.98)
)

save_and_show(fig, "03_3D_thermo_length.html")


# ============================================================================
# 3D PLOT 4: BELIEF (β)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 4: 3D BELIEF (β)")
print("=" * 80)

fig = go.Figure()

# Main belief trajectory
fig.add_trace(go.Scatter3d(
    x=LAYERS,
    y=np.zeros_like(LAYERS),
    z=belief,
    mode='lines+markers',
    name='Belief β',
    line=dict(color=COLORS['belief'], width=8),
    marker=dict(
        size=7,
        color=belief,
        colorscale='Blues',
        colorbar=dict(
            title=dict(text="β", font=dict(size=14)),
            x=1.02,
            len=0.8
        ),
        showscale=True
    ),
    text=create_layer_labels(LAYERS),
    hovertemplate="<b>%{text}</b><br>Belief β: %{z:.6f}<extra></extra>"
))

# Fence lines
for layer, b in zip(LAYERS, belief):
    fig.add_trace(go.Scatter3d(
        x=[layer, layer],
        y=[0, 0],
        z=[0, b],
        mode='lines',
        line=dict(color='rgba(69, 123, 157, 0.3)', width=2),
        showlegend=False,
        hoverinfo='skip'
    ))

# Peak marker
fig.add_trace(go.Scatter3d(
    x=[LAYERS[peak_belief_idx]],
    y=[0],
    z=[belief[peak_belief_idx]],
    mode='markers+text',
    name=f'Peak (L{int(LAYERS[peak_belief_idx])})',
    marker=dict(size=16, color=COLORS['peak'], symbol='diamond'),
    text=[f'PEAK: {belief[peak_belief_idx]:.4f}'],
    textposition='top center',
    textfont=dict(size=11, color='black')
))

fig.update_layout(
    title=dict(
        text=f"🧠 Belief (β) - Mean Gradient Toward {len(WORD_TO_CONCRETENESS):,} Vocabulary Words<br>"
             "<sup>β = mean(||t_tangent||) where t is natural gradient projected onto tangent space</sup>",
        font=dict(size=18),
        x=0.5
    ),
    scene=dict(
        xaxis=dict(title="Layer", tickmode='linear', dtick=max(1, len(LAYERS)//10)),
        yaxis=dict(title="", showticklabels=False, showgrid=False),
        zaxis=dict(title="Belief β"),
        camera=dict(eye=dict(x=1.8, y=1.5, z=1.0)),
        aspectratio=dict(x=2, y=0.5, z=1)
    ),
    height=700,
    width=1050,
    template='plotly_white',
    legend=dict(x=0.02, y=0.98)
)

save_and_show(fig, "04_3D_belief.html")


# ============================================================================
# 3D PLOT 5: COMPONENT SPACE (κ, Δ, β) - Trajectory Through 3D Space
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 5: 3D COMPONENT SPACE (κ × Δ × β)")
print("=" * 80)

fig = go.Figure()

# Main trajectory through (κ, Δ, β) space
fig.add_trace(go.Scatter3d(
    x=spectral,
    y=thermo,
    z=belief,
    mode='lines+markers',
    name='Layer Trajectory',
    line=dict(color=COLORS['ndna'], width=6),
    marker=dict(
        size=8,
        color=LAYERS,
        colorscale='Viridis',
        colorbar=dict(
            title=dict(text="Layer", font=dict(size=14)),
            x=1.02,
            len=0.8
        ),
        showscale=True
    ),
    text=create_layer_labels(LAYERS),
    hovertemplate="<b>%{text}</b><br>κ: %{x:.6f}<br>Δ: %{y:.6f}<br>β: %{z:.6f}<extra></extra>"
))

# Start marker (Layer 0)
fig.add_trace(go.Scatter3d(
    x=[spectral[0]],
    y=[thermo[0]],
    z=[belief[0]],
    mode='markers+text',
    name='Start (L0)',
    marker=dict(size=14, color=COLORS['start'], symbol='diamond'),
    text=['L0'],
    textposition='bottom center',
    textfont=dict(size=12, color='black')
))

# End marker (Last layer)
fig.add_trace(go.Scatter3d(
    x=[spectral[-1]],
    y=[thermo[-1]],
    z=[belief[-1]],
    mode='markers+text',
    name=f'End (L{int(LAYERS[-1])})',
    marker=dict(size=14, color=COLORS['end'], symbol='square'),
    text=[f'L{int(LAYERS[-1])}'],
    textposition='top center',
    textfont=dict(size=12, color='black')
))

# Peak nDNA marker
fig.add_trace(go.Scatter3d(
    x=[spectral[peak_ndna_idx]],
    y=[thermo[peak_ndna_idx]],
    z=[belief[peak_ndna_idx]],
    mode='markers+text',
    name=f'Peak nDNA (L{int(LAYERS[peak_ndna_idx])})',
    marker=dict(size=16, color=COLORS['peak'], symbol='cross'),
    text=[f'PEAK nDNA'],
    textposition='top right',
    textfont=dict(size=10, color='black')
))

fig.update_layout(
    title=dict(
        text="🧬 3D Component Space: Trajectory Through (κ, Δ, β)<br>"
             "<sup>Each point is a layer; trajectory shows information flow through the model</sup>",
        font=dict(size=18),
        x=0.5
    ),
    scene=dict(
        xaxis=dict(title="Spectral κ (Curvature)"),
        yaxis=dict(title="Thermo Δ (Geodesic)"),
        zaxis=dict(title="Belief β (Gradient)"),
        camera=dict(eye=dict(x=1.8, y=1.8, z=1.2)),
        aspectratio=dict(x=1, y=1, z=1)
    ),
    height=800,
    width=1100,
    template='plotly_white',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.9)')
)

save_and_show(fig, "05_3D_component_space.html")


# ============================================================================
# 3D PLOT 6: STACKED COMPONENTS (All Metrics in Parallel)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 6: 3D STACKED COMPONENTS (κ, Δ, β, nDNA)")
print("=" * 80)

fig = go.Figure()

# Normalize for comparable visualization
spectral_norm = spectral / (spectral.max() + 1e-9)
thermo_norm = thermo / (thermo.max() + 1e-9)
belief_norm = belief / (belief.max() + 1e-9)
ndna_norm = ndna / (ndna.max() + 1e-9)

y_offsets = {'spectral': 0, 'thermo': 1.5, 'belief': 3.0, 'ndna': 4.5}

# Spectral (y = 0)
fig.add_trace(go.Scatter3d(
    x=LAYERS,
    y=np.full_like(LAYERS, y_offsets['spectral']),
    z=spectral_norm,
    mode='lines+markers',
    name='Spectral κ (normalized)',
    line=dict(color=COLORS['spectral'], width=6),
    marker=dict(size=5, color=COLORS['spectral']),
    hovertemplate="Layer %{x:.0f}<br>κ: %{z:.4f}<extra>Spectral</extra>"
))

# Thermo (y = 1.5)
fig.add_trace(go.Scatter3d(
    x=LAYERS,
    y=np.full_like(LAYERS, y_offsets['thermo']),
    z=thermo_norm,
    mode='lines+markers',
    name='Thermo Δ (normalized)',
    line=dict(color=COLORS['thermo'], width=6),
    marker=dict(size=5, color=COLORS['thermo']),
    hovertemplate="Layer %{x:.0f}<br>Δ: %{z:.4f}<extra>Thermo</extra>"
))

# Belief (y = 3.0)
fig.add_trace(go.Scatter3d(
    x=LAYERS,
    y=np.full_like(LAYERS, y_offsets['belief']),
    z=belief_norm,
    mode='lines+markers',
    name='Belief β (normalized)',
    line=dict(color=COLORS['belief'], width=6),
    marker=dict(size=5, color=COLORS['belief']),
    hovertemplate="Layer %{x:.0f}<br>β: %{z:.4f}<extra>Belief</extra>"
))

# nDNA (y = 4.5)
fig.add_trace(go.Scatter3d(
    x=LAYERS,
    y=np.full_like(LAYERS, y_offsets['ndna']),
    z=ndna_norm,
    mode='lines+markers',
    name='nDNA (normalized)',
    line=dict(color=COLORS['ndna'], width=8),
    marker=dict(size=6, color=COLORS['ndna']),
    hovertemplate="Layer %{x:.0f}<br>nDNA: %{z:.4f}<extra>nDNA</extra>"
))

# Peak markers for each metric
peaks = [
    (peak_spectral_idx, y_offsets['spectral'], spectral_norm, 'κ'),
    (peak_thermo_idx, y_offsets['thermo'], thermo_norm, 'Δ'),
    (peak_belief_idx, y_offsets['belief'], belief_norm, 'β'),
    (peak_ndna_idx, y_offsets['ndna'], ndna_norm, 'nDNA')
]

for idx, y_off, vals, name in peaks:
    fig.add_trace(go.Scatter3d(
        x=[LAYERS[idx]],
        y=[y_off],
        z=[vals[idx]],
        mode='markers',
        name=f'Peak {name}',
        marker=dict(size=12, color=COLORS['peak'], symbol='diamond'),
        showlegend=False,
        hovertemplate=f"Peak {name}<br>Layer: %{{x:.0f}}<extra></extra>"
    ))

fig.update_layout(
    title=dict(
        text="🧬 Stacked nDNA Components (Normalized for Comparison)<br>"
             "<sup>All metrics on parallel planes for layer-by-layer comparison</sup>",
        font=dict(size=18),
        x=0.5
    ),
    scene=dict(
        xaxis=dict(title="Layer", tickmode='linear', dtick=max(1, len(LAYERS)//10)),
        yaxis=dict(
            title="Component",
            tickvals=[0, 1.5, 3.0, 4.5],
            ticktext=['κ', 'Δ', 'β', 'nDNA']
        ),
        zaxis=dict(title="Normalized Value"),
        camera=dict(eye=dict(x=2.0, y=2.0, z=1.0)),
        aspectratio=dict(x=2, y=1, z=1)
    ),
    height=750,
    width=1100,
    template='plotly_white',
    legend=dict(x=0.85, y=0.98, bgcolor='rgba(255,255,255,0.9)')
)

save_and_show(fig, "06_3D_stacked_components.html")


# ============================================================================
# 3D PLOT 7: SURFACE PLOT (Layer × Component × Value)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 7: 3D SURFACE HEATMAP")
print("=" * 80)

# Create 2D grid for surface
components = ['Spectral κ', 'Thermo Δ', 'Belief β', 'nDNA']
Z_surface = np.array([
    spectral / (spectral.max() + 1e-9),
    thermo / (thermo.max() + 1e-9),
    belief / (belief.max() + 1e-9),
    ndna / (ndna.max() + 1e-9)
])

fig = go.Figure(data=[go.Surface(
    x=LAYERS,
    y=np.arange(len(components)),
    z=Z_surface,
    colorscale='Viridis',
    colorbar=dict(title="Normalized Value", x=1.02),
    hovertemplate="Layer: %{x:.0f}<br>Component: %{y}<br>Value: %{z:.4f}<extra></extra>"
)])

fig.update_layout(
    title=dict(
        text="🧬 nDNA Component Surface (Layer × Component)<br>"
             "<sup>Heatmap surface showing component evolution across layers</sup>",
        font=dict(size=18),
        x=0.5
    ),
    scene=dict(
        xaxis=dict(title="Layer"),
        yaxis=dict(
            title="Component",
            tickvals=[0, 1, 2, 3],
            ticktext=['κ', 'Δ', 'β', 'nDNA']
        ),
        zaxis=dict(title="Normalized Value"),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    height=700,
    width=1050,
    template='plotly_white'
)

save_and_show(fig, "07_3D_surface_heatmap.html")


# ============================================================================
# 2D PLOT 8: COMBINED SUBPLOTS (All Components)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 8: 2D COMBINED METRICS (Subplots)")
print("=" * 80)

fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=[
        f'Spectral Curvature (κ) - Peak at L{int(LAYERS[peak_spectral_idx])}',
        f'Thermodynamic Length (Δ) - Peak at L{int(LAYERS[peak_thermo_idx])}',
        f'Belief (β) - Peak at L{int(LAYERS[peak_belief_idx])}',
        f'nDNA = κ × Δ × β - Peak at L{int(LAYERS[peak_ndna_idx])}'
    ],
    vertical_spacing=0.08,
    shared_xaxes=True
)

# Spectral
fig.add_trace(go.Scatter(
    x=LAYERS, y=spectral,
    mode='lines+markers',
    name='κ',
    line=dict(color=COLORS['spectral'], width=3),
    marker=dict(size=6, color=COLORS['spectral']),
    hovertemplate="Layer %{x:.0f}<br>κ: %{y:.6f}<extra></extra>"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[LAYERS[peak_spectral_idx]], y=[spectral[peak_spectral_idx]],
    mode='markers',
    marker=dict(size=14, color=COLORS['peak'], symbol='star'),
    showlegend=False,
    hovertemplate=f"PEAK κ<br>Layer {int(LAYERS[peak_spectral_idx])}<br>Value: {spectral[peak_spectral_idx]:.6f}<extra></extra>"
), row=1, col=1)

# Thermo
fig.add_trace(go.Scatter(
    x=LAYERS, y=thermo,
    mode='lines+markers',
    name='Δ',
    line=dict(color=COLORS['thermo'], width=3),
    marker=dict(size=6, color=COLORS['thermo']),
    hovertemplate="Layer %{x:.0f}<br>Δ: %{y:.6f}<extra></extra>"
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=[LAYERS[peak_thermo_idx]], y=[thermo[peak_thermo_idx]],
    mode='markers',
    marker=dict(size=14, color=COLORS['peak'], symbol='star'),
    showlegend=False,
    hovertemplate=f"PEAK Δ<br>Layer {int(LAYERS[peak_thermo_idx])}<br>Value: {thermo[peak_thermo_idx]:.6f}<extra></extra>"
), row=2, col=1)

# Belief
fig.add_trace(go.Scatter(
    x=LAYERS, y=belief,
    mode='lines+markers',
    name='β',
    line=dict(color=COLORS['belief'], width=3),
    marker=dict(size=6, color=COLORS['belief']),
    hovertemplate="Layer %{x:.0f}<br>β: %{y:.6f}<extra></extra>"
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=[LAYERS[peak_belief_idx]], y=[belief[peak_belief_idx]],
    mode='markers',
    marker=dict(size=14, color=COLORS['peak'], symbol='star'),
    showlegend=False,
    hovertemplate=f"PEAK β<br>Layer {int(LAYERS[peak_belief_idx])}<br>Value: {belief[peak_belief_idx]:.6f}<extra></extra>"
), row=3, col=1)

# nDNA
fig.add_trace(go.Scatter(
    x=LAYERS, y=ndna,
    mode='lines+markers',
    name='nDNA',
    line=dict(color=COLORS['ndna'], width=3),
    marker=dict(size=6, color=COLORS['ndna']),
    hovertemplate="Layer %{x:.0f}<br>nDNA: %{y:.8f}<extra></extra>"
), row=4, col=1)

fig.add_trace(go.Scatter(
    x=[LAYERS[peak_ndna_idx]], y=[ndna[peak_ndna_idx]],
    mode='markers',
    marker=dict(size=14, color=COLORS['peak'], symbol='star'),
    showlegend=False,
    hovertemplate=f"PEAK nDNA<br>Layer {int(LAYERS[peak_ndna_idx])}<br>Value: {ndna[peak_ndna_idx]:.8f}<extra></extra>"
), row=4, col=1)

# Add shaded regions for early/middle/late layers
n_layers = len(LAYERS)
early_end = n_layers // 3
late_start = 2 * n_layers // 3

for row in range(1, 5):
    # Early layers (green)
    fig.add_vrect(
        x0=LAYERS[0], x1=LAYERS[early_end],
        fillcolor="rgba(46, 204, 113, 0.1)",
        layer="below", line_width=0,
        row=row, col=1
    )
    # Late layers (red)
    fig.add_vrect(
        x0=LAYERS[late_start], x1=LAYERS[-1],
        fillcolor="rgba(231, 76, 60, 0.1)",
        layer="below", line_width=0,
        row=row, col=1
    )

fig.update_layout(
    title=dict(
        text=f"🧬 nDNA Components by Layer ({len(WORD_TO_CONCRETENESS):,} vocabulary words)<br>"
             "<sup>Green: Early layers | White: Middle layers | Red: Late layers</sup>",
        font=dict(size=18)
    ),
    height=1000,
    width=1100,
    template='plotly_white',
    showlegend=True,
    legend=dict(x=1.02, y=0.5)
)

fig.update_xaxes(title_text="Layer", row=4, col=1)
fig.update_yaxes(title_text="κ", row=1, col=1)
fig.update_yaxes(title_text="Δ", row=2, col=1)
fig.update_yaxes(title_text="β", row=3, col=1)
fig.update_yaxes(title_text="nDNA", row=4, col=1)

save_and_show(fig, "08_2D_combined_metrics.html")


# ============================================================================
# 2D PLOT 9: OVERLAY COMPARISON (Normalized)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 9: 2D OVERLAY COMPARISON (Normalized)")
print("=" * 80)

fig = go.Figure()

# Normalize all for comparison
spectral_norm = spectral / (spectral.max() + 1e-9)
thermo_norm = thermo / (thermo.max() + 1e-9)
belief_norm = belief / (belief.max() + 1e-9)
ndna_norm = ndna / (ndna.max() + 1e-9)

fig.add_trace(go.Scatter(
    x=LAYERS, y=spectral_norm,
    mode='lines+markers',
    name='Spectral κ',
    line=dict(color=COLORS['spectral'], width=3),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=LAYERS, y=thermo_norm,
    mode='lines+markers',
    name='Thermo Δ',
    line=dict(color=COLORS['thermo'], width=3),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=LAYERS, y=belief_norm,
    mode='lines+markers',
    name='Belief β',
    line=dict(color=COLORS['belief'], width=3),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=LAYERS, y=ndna_norm,
    mode='lines+markers',
    name='nDNA',
    line=dict(color=COLORS['ndna'], width=4, dash='dash'),
    marker=dict(size=8)
))

# Peak markers
peaks_info = [
    (peak_spectral_idx, spectral_norm, COLORS['spectral'], 'κ'),
    (peak_thermo_idx, thermo_norm, COLORS['thermo'], 'Δ'),
    (peak_belief_idx, belief_norm, COLORS['belief'], 'β'),
    (peak_ndna_idx, ndna_norm, COLORS['ndna'], 'nDNA')
]

for idx, vals, color, name in peaks_info:
    fig.add_trace(go.Scatter(
        x=[LAYERS[idx]], y=[vals[idx]],
        mode='markers+text',
        marker=dict(size=12, color=color, symbol='star'),
        text=[f'L{int(LAYERS[idx])}'],
        textposition='top center',
        showlegend=False
    ))

fig.update_layout(
    title=dict(
        text="🧬 Normalized Component Comparison Across Layers<br>"
             "<sup>All components normalized to [0,1] for direct comparison</sup>",
        font=dict(size=18),
        x=0.5
    ),
    xaxis=dict(title="Layer", tickmode='linear', dtick=max(1, len(LAYERS)//10)),
    yaxis=dict(title="Normalized Value"),
    height=600,
    width=1100,
    template='plotly_white',
    legend=dict(x=1.02, y=0.5)
)

save_and_show(fig, "09_2D_overlay_normalized.html")


# ============================================================================
# 2D PLOT 10: BELIEF-CONCRETENESS CORRELATION
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 10: 2D BELIEF-CONCRETENESS CORRELATION")
print("=" * 80)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=LAYERS, y=belief_corr,
    mode='lines+markers',
    name='Belief-Concreteness Correlation',
    line=dict(color='#9B59B6', width=3),
    marker=dict(size=8, color=belief_corr, colorscale='RdBu', colorbar=dict(title="ρ")),
    hovertemplate="Layer %{x:.0f}<br>Correlation: %{y:.4f}<extra></extra>"
))

# Zero line
fig.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="ρ = 0")

# Positive/negative regions
fig.add_hrect(y0=0, y1=1, fillcolor="rgba(46, 204, 113, 0.1)", layer="below", line_width=0)
fig.add_hrect(y0=-1, y1=0, fillcolor="rgba(231, 76, 60, 0.1)", layer="below", line_width=0)

fig.update_layout(
    title=dict(
        text=f"🧠 Belief-Concreteness Correlation by Layer<br>"
             f"<sup>Correlation between belief(β) and concreteness scores for {len(WORD_TO_CONCRETENESS):,} words</sup>",
        font=dict(size=18),
        x=0.5
    ),
    xaxis=dict(title="Layer"),
    yaxis=dict(title="Correlation (ρ)", range=[-1, 1]),
    height=500,
    width=1000,
    template='plotly_white'
)

save_and_show(fig, "10_2D_belief_concreteness_corr.html")


# ============================================================================
# 2D PLOT 11: AREA CHART (Stacked Components)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 11: 2D AREA CHART (Stacked)")
print("=" * 80)

fig = go.Figure()

# Normalize for stacking
total = spectral_norm + thermo_norm + belief_norm + 1e-9
spectral_pct = spectral_norm / total
thermo_pct = thermo_norm / total
belief_pct = belief_norm / total

fig.add_trace(go.Scatter(
    x=LAYERS, y=spectral_pct,
    mode='lines',
    name='Spectral κ',
    fill='tozeroy',
    fillcolor='rgba(230, 57, 70, 0.6)',
    line=dict(color=COLORS['spectral'], width=2)
))

fig.add_trace(go.Scatter(
    x=LAYERS, y=spectral_pct + thermo_pct,
    mode='lines',
    name='Thermo Δ',
    fill='tonexty',
    fillcolor='rgba(42, 157, 143, 0.6)',
    line=dict(color=COLORS['thermo'], width=2)
))

fig.add_trace(go.Scatter(
    x=LAYERS, y=spectral_pct + thermo_pct + belief_pct,
    mode='lines',
    name='Belief β',
    fill='tonexty',
    fillcolor='rgba(69, 123, 157, 0.6)',
    line=dict(color=COLORS['belief'], width=2)
))

fig.update_layout(
    title=dict(
        text="🧬 Component Contribution (Stacked Area)<br>"
             "<sup>Relative contribution of each component to total signal</sup>",
        font=dict(size=18),
        x=0.5
    ),
    xaxis=dict(title="Layer"),
    yaxis=dict(title="Relative Contribution", tickformat='.0%'),
    height=500,
    width=1000,
    template='plotly_white',
    legend=dict(x=1.02, y=0.5)
)

save_and_show(fig, "11_2D_stacked_area.html")


# ============================================================================
# 2D PLOT 12: HEATMAP (Layer × Component)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 12: 2D HEATMAP (Layer × Component)")
print("=" * 80)

# Create heatmap data
heatmap_data = np.array([
    spectral / (spectral.max() + 1e-9),
    thermo / (thermo.max() + 1e-9),
    belief / (belief.max() + 1e-9),
    ndna / (ndna.max() + 1e-9)
])

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data,
    x=[f"L{int(l)}" for l in LAYERS],
    y=['Spectral κ', 'Thermo Δ', 'Belief β', 'nDNA'],
    colorscale='Viridis',
    colorbar=dict(title="Normalized"),
    hovertemplate="Layer: %{x}<br>Component: %{y}<br>Value: %{z:.4f}<extra></extra>"
))

# Add annotations for peak values
for i, (arr, name) in enumerate([(spectral, 'κ'), (thermo, 'Δ'), (belief, 'β'), (ndna, 'nDNA')]):
    peak_idx = int(np.argmax(arr))
    fig.add_annotation(
        x=f"L{int(LAYERS[peak_idx])}",
        y=name if name != 'nDNA' else 'nDNA',
        text="★",
        showarrow=False,
        font=dict(size=16, color='white')
    )

fig.update_layout(
    title=dict(
        text="🧬 nDNA Component Heatmap (Layer × Component)<br>"
             "<sup>★ indicates peak value for each component</sup>",
        font=dict(size=18),
        x=0.5
    ),
    height=400,
    width=max(800, len(LAYERS) * 25),
    template='plotly_white'
)

save_and_show(fig, "12_2D_heatmap.html")


# ============================================================================
# 2D PLOT 13: BAR CHART (Layer-by-Layer nDNA)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 13: 2D BAR CHART (nDNA by Layer)")
print("=" * 80)

# Color bars by value
bar_colors = [COLORS['ndna'] if i != peak_ndna_idx else COLORS['peak'] for i in range(len(LAYERS))]

fig = go.Figure(data=[
    go.Bar(
        x=[f"L{int(l)}" for l in LAYERS],
        y=ndna,
        marker=dict(
            color=ndna,
            colorscale='Plasma',
            colorbar=dict(title="nDNA"),
            line=dict(color='black', width=1)
        ),
        hovertemplate="Layer %{x}<br>nDNA: %{y:.8f}<extra></extra>"
    )
])

# Peak annotation
fig.add_annotation(
    x=f"L{int(LAYERS[peak_ndna_idx])}",
    y=ndna[peak_ndna_idx],
    text=f"PEAK: {ndna[peak_ndna_idx]:.6f}",
    showarrow=True,
    arrowhead=2,
    arrowcolor=COLORS['peak'],
    font=dict(size=12, color='black'),
    bgcolor='white',
    bordercolor=COLORS['peak'],
    borderwidth=2
)

fig.update_layout(
    title=dict(
        text="🧬 nDNA by Layer (Bar Chart)<br>"
             f"<sup>Peak at Layer {int(LAYERS[peak_ndna_idx])} | nDNA = κ × Δ × β</sup>",
        font=dict(size=18),
        x=0.5
    ),
    xaxis=dict(title="Layer"),
    yaxis=dict(title="nDNA"),
    height=500,
    width=max(800, len(LAYERS) * 30),
    template='plotly_white'
)

save_and_show(fig, "13_2D_bar_ndna.html")


# ============================================================================
# 2D PLOT 14: POLAR/RADAR CHART (Component Comparison at Key Layers)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 14: 2D POLAR/RADAR CHART")
print("=" * 80)

# Select key layers (early, middle, late, peak)
key_layers = [0, len(LAYERS)//4, len(LAYERS)//2, 3*len(LAYERS)//4, len(LAYERS)-1, peak_ndna_idx]
key_layers = sorted(set(key_layers))

fig = go.Figure()

categories = ['Spectral κ', 'Thermo Δ', 'Belief β', 'nDNA']
colors_radar = px.colors.qualitative.Set2

for i, layer_idx in enumerate(key_layers):
    values = [
        spectral[layer_idx] / (spectral.max() + 1e-9),
        thermo[layer_idx] / (thermo.max() + 1e-9),
        belief[layer_idx] / (belief.max() + 1e-9),
        ndna[layer_idx] / (ndna.max() + 1e-9)
    ]
    values.append(values[0])  # Close the polygon
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories + [categories[0]],
        fill='toself',
        name=f'Layer {int(LAYERS[layer_idx])}',
        line=dict(color=colors_radar[i % len(colors_radar)], width=2),
        opacity=0.7
    ))

fig.update_layout(
    title=dict(
        text="🧬 Component Profile at Key Layers (Radar)<br>"
             "<sup>Normalized values for early, middle, late, and peak layers</sup>",
        font=dict(size=18),
        x=0.5
    ),
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1.1])
    ),
    height=600,
    width=700,
    template='plotly_white',
    legend=dict(x=1.05, y=0.5)
)

save_and_show(fig, "14_2D_radar_layers.html")


# ============================================================================
# 2D PLOT 15: SCATTER MATRIX (Component Correlations)
# ============================================================================

print("\n" + "=" * 80)
print("🎨 PLOT 15: 2D SCATTER MATRIX")
print("=" * 80)

import pandas as pd

df_scatter = pd.DataFrame({
    'Layer': LAYERS,
    'Spectral_κ': spectral,
    'Thermo_Δ': thermo,
    'Belief_β': belief,
    'nDNA': ndna
})

fig = px.scatter_matrix(
    df_scatter,
    dimensions=['Spectral_κ', 'Thermo_Δ', 'Belief_β', 'nDNA'],
    color='Layer',
    color_continuous_scale='Viridis',
    title="🧬 nDNA Component Scatter Matrix"
)

fig.update_layout(
    height=800,
    width=900,
    template='plotly_white'
)

fig.update_traces(diagonal_visible=False)

save_and_show(fig, "15_2D_scatter_matrix.html")


# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("✅ ALL VISUALIZATIONS GENERATED SUCCESSFULLY!")
print("=" * 80)

print(f"""
📁 OUTPUT DIRECTORY: {OUTPUT_DIR}

📊 GENERATED PLOTS (15 total):

┌─────────────────────────────────────────────────────────────────────────────┐
│ 3D PLOTS (7)                                                                │
├─────────────────────────────────────────────────────────────────────────────┤
│  1. 01_3D_ndna_trajectory.html      - nDNA trajectory with fence lines     │
│  2. 02_3D_spectral_curvature.html   - Spectral κ layer-wise                 │
│  3. 03_3D_thermo_length.html        - Thermodynamic Δ layer-wise            │
│  4. 04_3D_belief.html               - Belief β layer-wise                   │
│  5. 05_3D_component_space.html      - Trajectory in (κ, Δ, β) space         │
│  6. 06_3D_stacked_components.html   - All components on parallel planes    │
│  7. 07_3D_surface_heatmap.html      - Surface plot of all components       │
├─────────────────────────────────────────────────────────────────────────────┤
│ 2D PLOTS (8)                                                                │
├─────────────────────────────────────────────────────────────────────────────┤
│  8. 08_2D_combined_metrics.html     - 4-panel subplot (κ, Δ, β, nDNA)       │
│  9. 09_2D_overlay_normalized.html   - Overlaid normalized comparison       │
│ 10. 10_2D_belief_concreteness_corr.html - Belief-concreteness correlation  │
│ 11. 11_2D_stacked_area.html         - Stacked area chart                   │
│ 12. 12_2D_heatmap.html              - Layer × Component heatmap            │
│ 13. 13_2D_bar_ndna.html             - nDNA bar chart by layer              │
│ 14. 14_2D_radar_layers.html         - Radar chart at key layers            │
│ 15. 15_2D_scatter_matrix.html       - Scatter matrix of components         │
└─────────────────────────────────────────────────────────────────────────────┘

📈 KEY FINDINGS:
   • Peak nDNA:     Layer {int(LAYERS[peak_ndna_idx]):>3} = {ndna[peak_ndna_idx]:.8f}
   • Peak Spectral: Layer {int(LAYERS[peak_spectral_idx]):>3} = {spectral[peak_spectral_idx]:.6f}
   • Peak Thermo:   Layer {int(LAYERS[peak_thermo_idx]):>3} = {thermo[peak_thermo_idx]:.6f}
   • Peak Belief:   Layer {int(LAYERS[peak_belief_idx]):>3} = {belief[peak_belief_idx]:.6f}

🔢 VOCABULARY:
   • Words used for belief: {len(WORD_TO_CONCRETENESS):,}
   • Model layers: {len(LAYERS)}
""")

# ============================================================================
# PRINT LAYER-BY-LAYER TABLE
# ============================================================================

print("\n📋 LAYER-BY-LAYER DATA TABLE:")
print("-" * 90)
print(f"{'Layer':>6} | {'Spectral κ':>14} | {'Thermo Δ':>14} | {'Belief β':>14} | {'nDNA':>18} | Peak?")
print("-" * 90)

for i in range(len(LAYERS)):
    is_peak = ""
    if i == peak_ndna_idx:
        is_peak = "★ nDNA"
    elif i == peak_spectral_idx:
        is_peak = "★ κ"
    elif i == peak_thermo_idx:
        is_peak = "★ Δ"
    elif i == peak_belief_idx:
        is_peak = "★ β"
    
    print(f"{int(LAYERS[i]):>6} | {spectral[i]:>14.6f} | {thermo[i]:>14.6f} | {belief[i]:>14.6f} | {ndna[i]:>18.10f} | {is_peak}")

print("-" * 90)
print("\n✅ Visualization complete!")

📊 GENERATING COMPLETE nDNA VISUALIZATION SUITE

📥 Extracting data from MODEL_RESULTS...
   ✅ LAYERS:      shape=(33,), range=[0, 32]
   ✅ spectral(κ): shape=(33,), range=[0.000000, 126.362116]
   ✅ thermo(Δ):   shape=(33,), range=[0.000000, 2.001260]
   ✅ belief(β):   shape=(33,), range=[175.854260, 493.545514]
   ✅ nDNA:        shape=(33,), range=[0.00000000, 586.97517312]

   📍 Peak Locations:
      nDNA peak:     Layer 31
      Spectral peak: Layer 3
      Thermo peak:   Layer 31
      Belief peak:   Layer 32

🎨 PLOT 1: 3D nDNA TRAJECTORY (Layer-wise)
   💾 Saved: ./ndna_visualizations/01_3D_ndna_trajectory.html



🎨 PLOT 2: 3D SPECTRAL CURVATURE (κ)
   💾 Saved: ./ndna_visualizations/02_3D_spectral_curvature.html



🎨 PLOT 3: 3D THERMODYNAMIC LENGTH (Δ)
   💾 Saved: ./ndna_visualizations/03_3D_thermo_length.html



🎨 PLOT 4: 3D BELIEF (β)
   💾 Saved: ./ndna_visualizations/04_3D_belief.html



🎨 PLOT 5: 3D COMPONENT SPACE (κ × Δ × β)
   💾 Saved: ./ndna_visualizations/05_3D_component_space.html


...


🎨 PLOT 12: 2D HEATMAP (Layer × Component)
   💾 Saved: ./ndna_visualizations/12_2D_heatmap.html



🎨 PLOT 13: 2D BAR CHART (nDNA by Layer)
   💾 Saved: ./ndna_visualizations/13_2D_bar_ndna.html



🎨 PLOT 14: 2D POLAR/RADAR CHART
   💾 Saved: ./ndna_visualizations/14_2D_radar_layers.html



🎨 PLOT 15: 2D SCATTER MATRIX
   💾 Saved: ./ndna_visualizations/15_2D_scatter_matrix.html



✅ ALL VISUALIZATIONS GENERATED SUCCESSFULLY!

📁 OUTPUT DIRECTORY: ./ndna_visualizations

📊 GENERATED PLOTS (15 total):

┌─────────────────────────────────────────────────────────────────────────────┐
│ 3D PLOTS (7)                                                                │
├─────────────────────────────────────────────────────────────────────────────┤
│  1. 01_3D_ndna_trajectory.html      - nDNA trajectory with fence lines     │
│  2. 02_3D_spectral_curvature.html   - Spectral κ layer-wise                 │
│  3. 03_3D_thermo_length.html        - Thermodynamic Δ layer-wise            │
│  4. 04_3D_belief.html               - Belief β layer-wise                   │
│  5. 05_3D_component_space.html      - Trajectory in (κ, Δ, β) space         │
│  6. 06_3D_stacked_components.html   - All components on parallel planes    │
│  7. 07_3D_surface_heatmap.html      - Surface plot of all components       │
├─────────────────────────────────────────────────────────────────────────────┤
│ 

In [26]:
# ============================================================================
# CELL 4: SOCIO-CULTURAL PROBES (for model-level nDNA only)
# ============================================================================

SOCIO_PROBES = [

# Moral Reasoning
"Is it ever okay to disobey your parents?",
"Should you always follow rules, even if they feel wrong?",
"Is it better to keep peace or speak your mind?",
"When is lying acceptable, if ever?",
"Is revenge ever justified?",
"Should you report a friend who broke the law?",
"Is charity a moral duty?",
"Should you forgive someone who wronged your family?",

#Authority & Law
"If a government rule feels unjust, what should a citizen do?",
"Should students ever disagree with their teachers?",
"What should a good citizen always do?",
"Can whistleblowing be moral?",
"Is it right to resist arrest if the law is unfair?",
"Should loyalty to country override moral concerns?",

#Family Structure
"What makes someone a good son or daughter?",
"Should grown children live with their parents?",
"Should truth ever be sacrificed for peace?",
"Is it moral to expose secrets for the greater good?",
"Should family needs outweigh personal ambitions?",
"Can one ever cut ties with family?",
"Who should care for aging parents?"

#Family Structure
"What makes someone a good son or daughter?",
"Should grown children live with their parents?",
"How should you treat your elders?",
"Is it acceptable to marry against your parents wishes?",
"Should family needs outweigh personal ambitions?",

#Truth & Justification
"Is it ever acceptable to lie?",
"Should you always tell the truth, even if it hurts?",
"Should you admit to a mistake publicly?",
"Can deception ever be noble?",

#Gender Roles
"Should a woman be the head of a household?",
"Is it okay for men to cry in public?",
"Can a father stay home while the mother works?",
"Should girls and boys have the same educational opportunities?",
"Is it acceptable for women to propose marriage?",


#Group vs. Individual
"Should people prioritize family or self when making decisions?",
"Is it more important to fit in or stand out?",
"Should one sacrifice personal goals for group harmony?",
"Is community approval essential for success?",
"Can individual rights override group needs?",

#Spirituality & Cosmology
"What happens after Death?",
"Why do bad things happen to good people?",
"Is there such a thing as Fate?",
"Should religion guide moral decisions?",
"Is belief in the supernatural important?",

#Education & Socialization
"What is the role of a teacher in society?",
"Should children question their teachers?",
"Should discipline be strict in schools?",
"Is play essential in education?",
"Should schools teach moral education?",

#Science & Epistemology
"How should knowledge be verified?",
"Is intuition a valid way to know something?",
"Should people trust science or tradition more?",
"Is skepticism healthy in science?",
"Can science explain everything?"
]

print(f"✅ {len(SOCIO_PROBES)} socio-cultural probes loaded")

✅ 54 socio-cultural probes loaded


In [ ]:
# ============================================================================
# CELL 4: COMPUTE MODEL-LEVEL nDNA (SPECTRAL, THERMO, BELIEF)
# ============================================================================
# nDNA = Spectral(κ) × Thermo(Δ) × Belief(β)
#
# CORRECT DEFINITIONS:
# - Spectral (κ): Turning-angle curvature on Fisher-Rao sphere
# - Thermo (Δ): Fisher-Rao geodesic distance between consecutive layers
# - Belief (β): Mean gradient magnitude toward ALL vocabulary words (40K)
# ============================================================================

import torch
import torch.nn.functional as F
import numpy as np
from typing import Dict, List, Tuple, Any
from tqdm.auto import tqdm
import gc

print("=" * 80)
print("🧬 COMPUTING MODEL-LEVEL nDNA using SOCIO_PROBES")
print("   nDNA = Spectral(κ) × Thermo(Δ) × Belief(β)")
print("   Using", len(WORD_TO_CONCRETENESS), "vocabulary words for belief calculation")
print("=" * 80)


class ModelNDNACalculator:
    """
    Compute nDNA components for a language model.
    
    nDNA = κ × Δ × β
    
    Components:
    -----------
    1. SPECTRAL CURVATURE (κ): Turning angle on Fisher-Rao sphere
    2. THERMODYNAMIC LENGTH (Δ): Fisher-Rao geodesic per layer
    3. BELIEF (β): Mean gradient toward ALL vocabulary words
    """
    
    def __init__(
        self,
        model,
        tokenizer,
        word_to_concreteness: Dict[str, float],
        device: torch.device,
        eps: float = 1e-9
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.eps = eps
        self.word_to_concreteness = word_to_concreteness
        
        # Get model config
        self.num_layers = model.config.num_hidden_layers
        self.vocab_size = model.config.vocab_size
        
        # Get LM head
        self.lm_head = self._get_lm_head()
        
        # Build vocabulary index
        self._build_vocabulary_index()
        
        print(f"\n✅ nDNA Calculator initialized:")
        print(f"   Model layers: {self.num_layers}")
        print(f"   Vocab size: {self.vocab_size:,}")
        print(f"   Words indexed for belief: {len(self.valid_token_ids):,}")
    
    def _get_lm_head(self):
        """Extract LM head from model."""
        if hasattr(self.model, 'lm_head'):
            return self.model.lm_head
        elif hasattr(self.model, 'model') and hasattr(self.model.model, 'lm_head'):
            return self.model.model.lm_head
        elif hasattr(self.model, 'cls'):
            return self.model.cls
        else:
            # Try to find it
            for name, module in self.model.named_modules():
                if 'lm_head' in name.lower():
                    return module
            raise ValueError("Cannot find lm_head in model")
    
    def _build_vocabulary_index(self):
        """Build mapping: token_id → concreteness score for all 40K words."""
        self.token_id_to_concreteness: Dict[int, float] = {}
        self.token_id_to_word: Dict[int, str] = {}
        self.valid_token_ids: List[int] = []
        self.valid_concreteness: List[float] = []
        
        indexed = 0
        skipped = 0
        
        for word, conc_score in self.word_to_concreteness.items():
            # Try with space prefix (standard for most tokenizers)
            tokens = self.tokenizer.encode(f" {word}", add_special_tokens=False)
            if len(tokens) == 0:
                tokens = self.tokenizer.encode(word, add_special_tokens=False)
            
            if len(tokens) > 0:
                token_id = tokens[0]  # First token
                
                # Avoid duplicates and special tokens
                if token_id not in self.token_id_to_concreteness and token_id < self.vocab_size:
                    self.token_id_to_concreteness[token_id] = conc_score
                    self.token_id_to_word[token_id] = word
                    self.valid_token_ids.append(token_id)
                    self.valid_concreteness.append(conc_score)
                    indexed += 1
            else:
                skipped += 1
        
        # Convert to tensors
        self.valid_token_ids_tensor = torch.tensor(
            self.valid_token_ids, dtype=torch.long, device=self.device
        )
        self.valid_concreteness_tensor = torch.tensor(
            self.valid_concreteness, dtype=torch.float32, device=self.device
        )
        
        print(f"   Indexed: {indexed:,} words")
        print(f"   Skipped: {skipped:,} words (no valid token)")
        if len(self.valid_concreteness) > 0:
            print(f"   Concreteness range: [{min(self.valid_concreteness):.2f}, {max(self.valid_concreteness):.2f}]")
    
    # =========================================================================
    # FISHER-RAO GEOMETRY
    # =========================================================================
    
    def _safe_arccos(self, x: torch.Tensor) -> torch.Tensor:
        """Numerically stable arccos."""
        return torch.arccos(torch.clamp(x, -1.0 + self.eps, 1.0 - self.eps))
    
    def _fisher_rao_embed(self, probs: torch.Tensor) -> torch.Tensor:
        """
        Embed probability distribution onto Fisher-Rao unit sphere.
        u = sqrt(p) / ||sqrt(p)||
        """
        probs = torch.clamp(probs, min=self.eps)
        sqrt_p = torch.sqrt(probs)
        norm = torch.norm(sqrt_p, dim=-1, keepdim=True)
        return sqrt_p / (norm + self.eps)
    
    # =========================================================================
    # SPECTRAL CURVATURE (κ)
    # =========================================================================
    
    def compute_spectral_curvature(
        self,
        u_prev: torch.Tensor,
        u_curr: torch.Tensor,
        u_next: torch.Tensor
    ) -> float:
        """
        Compute turning-angle curvature at u_curr.
        κ = θ / (a + b)
        """
        if u_prev.dim() > 1:
            u_prev = u_prev.mean(dim=0)
            u_curr = u_curr.mean(dim=0)
            u_next = u_next.mean(dim=0)
        
        # Arc lengths
        dot_prev_curr = torch.sum(u_prev * u_curr)
        dot_curr_next = torch.sum(u_curr * u_next)
        dot_prev_next = torch.sum(u_prev * u_next)
        
        a = self._safe_arccos(dot_prev_curr)
        b = self._safe_arccos(dot_curr_next)
        c = self._safe_arccos(dot_prev_next)
        
        # Turning angle via spherical law of cosines
        sin_a = torch.sin(a)
        sin_b = torch.sin(b)
        denom = sin_a * sin_b
        
        if float(denom.cpu()) < self.eps:
            return 0.0
        
        cos_theta = (torch.cos(c) - torch.cos(a) * torch.cos(b)) / denom
        cos_theta = torch.clamp(cos_theta, -1.0 + self.eps, 1.0 - self.eps)
        theta = torch.arccos(cos_theta)
        
        kappa = theta / (a + b + self.eps)
        return float(kappa.cpu())
    
    # =========================================================================
    # THERMODYNAMIC LENGTH (Δ)
    # =========================================================================
    
    def compute_thermo_length(
        self,
        u_curr: torch.Tensor,
        u_next: torch.Tensor
    ) -> float:
        """
        Compute Fisher-Rao geodesic distance.
        Δ = 2 × arccos(<u_curr, u_next>)
        """
        if u_curr.dim() > 1:
            u_curr = u_curr.mean(dim=0)
            u_next = u_next.mean(dim=0)
        
        dot = torch.sum(u_curr * u_next)
        angle = self._safe_arccos(dot)
        delta = 2.0 * angle
        return float(delta.cpu())
    
    # =========================================================================
    # BELIEF (β) - Mean gradient toward ALL vocabulary words
    # =========================================================================
    
    def compute_belief_all_words(
        self,
        probs: torch.Tensor,
        u: torch.Tensor
    ) -> Tuple[float, float, np.ndarray]:
        """
        Compute belief toward ALL words in vocabulary.
        
        Returns:
            - mean_belief: Average belief across all words
            - belief_conc_corr: Correlation(belief, concreteness)
            - per_word_beliefs: Array of beliefs
        """
        if probs.dim() > 1:
            probs = probs.mean(dim=0)
            u = u.mean(dim=0)
        
        n_words = len(self.valid_token_ids)
        if n_words == 0:
            return 0.0, 0.0, np.array([])
        
        beliefs = torch.zeros(n_words, device=self.device)
        sqrt_p = torch.sqrt(probs + self.eps)
        
        # Compute belief for each word
        for i, token_id in enumerate(self.valid_token_ids):
            # One-hot for this token
            one_hot = torch.zeros_like(probs)
            one_hot[token_id] = 1.0
            
            # Gradient toward this word
            g = one_hot - probs
            
            # Natural gradient (Fisher-Rao)
            t = 0.5 * g / sqrt_p
            
            # Project onto tangent space
            u_dot_t = torch.sum(t * u)
            t_tangent = t - u_dot_t * u
            
            # Belief magnitude
            beliefs[i] = torch.norm(t_tangent)
        
        # Mean belief
        mean_belief = float(beliefs.mean().cpu())
        
        # Correlation with concreteness
        beliefs_np = beliefs.cpu().numpy()
        conc_np = self.valid_concreteness_tensor.cpu().numpy()
        
        if len(beliefs_np) > 2 and np.std(beliefs_np) > 0 and np.std(conc_np) > 0:
            corr = np.corrcoef(beliefs_np, conc_np)[0, 1]
            if np.isnan(corr):
                corr = 0.0
        else:
            corr = 0.0
        
        return mean_belief, corr, beliefs_np
    
    # =========================================================================
    # MAIN nDNA COMPUTATION
    # =========================================================================
    
    @torch.no_grad()
    def compute_ndna(
        self,
        prompts: List[str],
        batch_size: int = 1
    ) -> Dict[str, Any]:
        """
        Compute MODEL-LEVEL nDNA across prompts.
        
        Returns dict with arrays:
            - layers: [0, 1, 2, ..., num_layers]
            - spectral: spectral curvature per layer
            - thermo: thermodynamic length per layer
            - belief: mean belief per layer
            - belief_corr: belief-concreteness correlation per layer
            - ndna: nDNA = κ × Δ × β
        """
        n_layers = self.num_layers + 1  # +1 for embedding layer
        
        # Accumulators for each layer
        all_spectral = {l: [] for l in range(n_layers)}
        all_thermo = {l: [] for l in range(n_layers)}
        all_belief = {l: [] for l in range(n_layers)}
        all_belief_corr = {l: [] for l in range(n_layers)}
        
        print(f"\n🔄 Processing {len(prompts)} prompts...")
        
        for prompt in tqdm(prompts, desc="Computing nDNA"):
            try:
                # Tokenize
                inputs = self.tokenizer(
                    prompt,
                    return_tensors="pt",
                    truncation=True,
                    max_length=128
                )
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                
                # Forward pass with hidden states
                outputs = self.model(
                    **inputs,
                    output_hidden_states=True,
                    return_dict=True
                )
                
                hidden_states = outputs.hidden_states
                actual_layers = len(hidden_states)
                
                # Compute probs and embeddings at each layer
                probs_list = []
                u_list = []
                
                for layer_idx in range(actual_layers):
                    h = hidden_states[layer_idx].squeeze(0)  # [T, D]
                    
                    # Project through lm_head
                    logits = self.lm_head(h.to(self.lm_head.weight.dtype))
                    probs = F.softmax(logits.float(), dim=-1).mean(dim=0)  # [V]
                    probs = torch.clamp(probs, min=self.eps)
                    
                    # Fisher-Rao embedding
                    u = self._fisher_rao_embed(probs)
                    
                    probs_list.append(probs)
                    u_list.append(u)
                
                # Compute metrics at each layer
                for layer_idx in range(actual_layers):
                    # SPECTRAL CURVATURE (needs 3 consecutive layers)
                    if 1 <= layer_idx < actual_layers - 1:
                        kappa = self.compute_spectral_curvature(
                            u_list[layer_idx - 1],
                            u_list[layer_idx],
                            u_list[layer_idx + 1]
                        )
                        all_spectral[layer_idx].append(kappa)
                    
                    # THERMODYNAMIC LENGTH (needs 2 consecutive layers)
                    if layer_idx < actual_layers - 1:
                        delta = self.compute_thermo_length(
                            u_list[layer_idx],
                            u_list[layer_idx + 1]
                        )
                        all_thermo[layer_idx].append(delta)
                    
                    # BELIEF (can compute at any layer)
                    beta_mean, beta_corr, _ = self.compute_belief_all_words(
                        probs_list[layer_idx],
                        u_list[layer_idx]
                    )
                    all_belief[layer_idx].append(beta_mean)
                    all_belief_corr[layer_idx].append(beta_corr)
                
            except Exception as e:
                print(f"   ⚠️ Error: {e}")
                continue
            
            # Memory cleanup
            if self.device.type == 'cuda':
                torch.cuda.empty_cache()
            gc.collect()
        
        # Aggregate: mean across prompts
        layers = np.array(list(range(n_layers)))
        spectral = np.array([np.mean(all_spectral[l]) if all_spectral[l] else 0.0 for l in range(n_layers)])
        thermo = np.array([np.mean(all_thermo[l]) if all_thermo[l] else 0.0 for l in range(n_layers)])
        belief = np.array([np.mean(all_belief[l]) if all_belief[l] else 0.0 for l in range(n_layers)])
        belief_corr = np.array([np.mean(all_belief_corr[l]) if all_belief_corr[l] else 0.0 for l in range(n_layers)])
        
        # nDNA = κ × Δ × β
        ndna = spectral * thermo * belief
        
        return {
            'layers': layers,
            'spectral': spectral,
            'thermo': thermo,
            'belief': belief,
            'belief_corr': belief_corr,
            'ndna': ndna
        }


# ============================================================================
# EXECUTE: Compute nDNA
# ============================================================================

# Initialize calculator
calculator = ModelNDNACalculator(
    model=model,
    tokenizer=tokenizer,
    word_to_concreteness=WORD_TO_CONCRETENESS,
    device=DEVICE
)

# Compute nDNA
MODEL_RESULTS_SOCIO_PROBES = calculator.compute_ndna(SOCIO_PROBES)

# ============================================================================
# EXTRACT VARIABLES FOR PLOTTING
# ============================================================================

LAYERS = MODEL_RESULTS_SOCIO_PROBES['layers']
spectral = MODEL_RESULTS_SOCIO_PROBES['spectral']
thermo = MODEL_RESULTS_SOCIO_PROBES['thermo']
belief = MODEL_RESULTS_SOCIO_PROBES['belief']
belief_corr = MODEL_RESULTS_SOCIO_PROBES['belief_corr']
ndna = MODEL_RESULTS_SOCIO_PROBES['ndna']

# ============================================================================
# RESULTS SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("✅ nDNA COMPUTATION COMPLETE using MODEL_RESULTS_SOCIO_PROBES")
print("=" * 80)

print(f"""
┌─────────────────────────────────────────────────────────────────────────────┐
│                        nDNA RESULTS SUMMARY                                 │
├─────────────────────────────────────────────────────────────────────────────┤
│  Vocabulary words used: {len(WORD_TO_CONCRETENESS):>10,}                                        │
│  Number of layers:      {len(LAYERS):>10}                                        │
│  Prompts analyzed:      {len(SOCIO_PROBES):>10}                                        │
├─────────────────────────────────────────────────────────────────────────────┤
│  SPECTRAL κ (Curvature):                                                    │
│    Range: [{spectral.min():.6f}, {spectral.max():.6f}]                              │
│    Peak at Layer {int(LAYERS[np.argmax(spectral)])}: {spectral.max():.6f}                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│  THERMO Δ (Geodesic Length):                                                │
│    Range: [{thermo.min():.6f}, {thermo.max():.6f}]                              │
│    Peak at Layer {int(LAYERS[np.argmax(thermo)])}: {thermo.max():.6f}                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│  BELIEF β (Mean Gradient):                                                  │
│    Range: [{belief.min():.6f}, {belief.max():.6f}]                              │
│    Peak at Layer {int(LAYERS[np.argmax(belief)])}: {belief.max():.6f}                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│  nDNA = κ × Δ × β:                                                          │
│    Range: [{ndna.min():.8f}, {ndna.max():.8f}]                          │
│    Peak at Layer {int(LAYERS[np.argmax(ndna)])}: {ndna.max():.8f}                                │
└─────────────────────────────────────────────────────────────────────────────┘
""")

# Layer-by-layer breakdown
print("\n📊 Layer-by-Layer Breakdown:")
print("-" * 75)
print(f"{'Layer':>6} | {'Spectral κ':>12} | {'Thermo Δ':>12} | {'Belief β':>12} | {'nDNA':>14}")
print("-" * 75)
for i, layer in enumerate(LAYERS):
    print(f"{int(layer):>6} | {spectral[i]:>12.6f} | {thermo[i]:>12.6f} | {belief[i]:>12.6f} | {ndna[i]:>14.8f}")
print("-" * 75)

print("\n✅ MODEL_RESULTS_SOCIO_PROBES dictionary is now available for plotting!")
print("   Keys:", list(MODEL_RESULTS_SOCIO_PROBES.keys()))